In [ ]:
pip install pandas numpy tqdm scikit-learn umap-learn matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

# ==========================================
# 1. SETUP PATHS & CONFIG
# ==========================================
PROJECT_ROOT = "E:\groteEdeepprofilerdingen\deepprofileroutputsetc\my_dp_project (Copy)"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs/results/features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs/metadata/index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
REG_PARAM = 1e-2

# ==========================================
# 2. HIERARCHICAL DATA LOADING
# ==========================================
print("Loading Metadata...")
meta = pd.read_csv(METADATA_PATH)

site_level_data = []
site_level_features = []

print(f"Loading features from {FEATURES_BASE}...")
for i in tqdm(meta.index, desc="Sites Processed"):
    filename = os.path.join(
        FEATURES_BASE,
        str(meta.loc[i, "Metadata_Plate"]),
        str(meta.loc[i, "Metadata_Well"]),
        f"{meta.loc[i, 'Metadata_Site']}.npz"
    )

    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    # Aggregation 1: Median per site
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Create Main DataFrame
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)

# ==========================================
# 3. WELL-LEVEL AGGREGATION (MEAN)
# ==========================================
# We keep wells separate to see the red no_sgRNA dots individually
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 4. SPHERING (ZCA WHITENING)
# ==========================================
def perform_sphering(df, ctrl_label, reg):
    ctrl_wells = df[df["Treatment"] == ctrl_label][feature_cols].values
    if len(ctrl_wells) < 2:
        print("Warning: Need at least 2 control wells for sphering.")
        return None
    
    mean_vec = np.mean(ctrl_wells, axis=0)
    X_centered = df[feature_cols].values - mean_vec
    
    # Calculate ZCA Matrix
    cov = np.dot((ctrl_wells - mean_vec).T, (ctrl_wells - mean_vec)) / (len(ctrl_wells) - 1)
    evals, evecs = np.linalg.eigh(cov + reg * np.eye(cov.shape[0]))
    zca_matrix = np.dot(evecs, np.dot(np.diag(1.0 / np.sqrt(evals + 1e-6)), evecs.T))
    
    return np.dot(X_centered, zca_matrix.T)

# Prepare both datasets
X_raw_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
X_sphered = perform_sphering(wells, CONTROL_NAME, REG_PARAM)

# ==========================================
# 5. UMAP & VISUALIZATION (RED CONTROLS)
# ==========================================
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(22, 10))
n_neighbors = min(15, len(wells) - 1)
reducer = umap.UMAP(n_neighbors=n_neighbors, random_state=42)

# Create color palette: all Treatments are 'turbo', but CONTROL_NAME is 'red'
unique_treats = wells["Treatment"].unique()
palette = sns.color_palette("turbo", n_colors=len(unique_treats))
color_map = dict(zip(unique_treats, palette))
color_map[CONTROL_NAME] = "red" 

# Plot 1: Non-Sphered
print("Running UMAP on Raw data...")
emb_raw = reducer.fit_transform(X_raw_scaled)
sns.scatterplot(x=emb_raw[:, 0], y=emb_raw[:, 1], hue=wells["Treatment"], 
                palette=color_map, style=(wells["Treatment"] == CONTROL_NAME), 
                markers={True: "X", False: "o"}, s=180, ax=ax1, edgecolor='k')
ax1.set_title("Well-Level: NO SPHERING (Red X = Control)")

# Plot 2: Sphered
if X_sphered is not None:
    print("Running UMAP on Sphered data...")
    emb_sph = reducer.fit_transform(X_sphered)
    sns.scatterplot(x=emb_sph[:, 0], y=emb_sph[:, 1], hue=wells["Treatment"], 
                    palette=color_map, style=(wells["Treatment"] == CONTROL_NAME), 
                    markers={True: "X", False: "o"}, s=180, ax=ax2, edgecolor='k')
    ax2.set_title(f"Well-Level: SPHERED (Red X = Control)")

# Legend Handling
ax1.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax2.get_legend().remove() # Redundant legend

plt.tight_layout()
plt.show()

In [ ]:
pip install plotly

In [ ]:
pip install nbformat

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ==========================================
# 1. SETUP PATHS & CONFIG (Windows Style)
# ==========================================
# Using 'r' before the string prevents "invalid escape sequence" errors on Windows
PROJECT_ROOT = r"E:\groteEdeepprofilerdingen\deepprofileroutputsetc\my_dp_project (Copy)"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
REG_PARAM = 1e-2

# ==========================================
# 2. HIERARCHICAL DATA LOADING
# ==========================================
print("Loading Metadata...")
if not os.path.exists(METADATA_PATH):
    raise FileNotFoundError(f"Metadata not found at {METADATA_PATH}")

meta = pd.read_csv(METADATA_PATH)
site_level_data = []
site_level_features = []

print(f"Loading features from: {FEATURES_BASE}")
for i in tqdm(meta.index, desc="Processing Sites"):
    filename = os.path.join(
        FEATURES_BASE,
        str(meta.loc[i, "Metadata_Plate"]),
        str(meta.loc[i, "Metadata_Well"]),
        f"{meta.loc[i, 'Metadata_Site']}.npz"
    )

    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    # AGGREGATION 1: Site-level Median
                    site_median = np.median(cells_f, axis=0)
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(site_median)
        except: continue

# Create DataFrame
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)

# ==========================================
# 3. WELL-LEVEL AGGREGATION (MEAN)
# ==========================================
# This allows us to see individual dots for each well
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()
print(f"Done. Found {len(wells)} unique wells.")

# ==========================================
# 4. SPHERING (ZCA WHITENING)
# ==========================================
def perform_sphering(df, ctrl_label, reg):
    ctrl_wells = df[df["Treatment"] == ctrl_label][feature_cols].values
    if len(ctrl_wells) < 2:
        print("Warning: Need at least 2 control wells for sphering.")
        return None
    
    mean_vec = np.mean(ctrl_wells, axis=0)
    X_centered = df[feature_cols].values - mean_vec
    
    # Calculate ZCA Matrix
    cov = np.dot((ctrl_wells - mean_vec).T, (ctrl_wells - mean_vec)) / (len(ctrl_wells) - 1)
    evals, evecs = np.linalg.eigh(cov + reg * np.eye(cov.shape[0]))
    zca_matrix = np.dot(evecs, np.dot(np.diag(1.0 / np.sqrt(evals + 1e-6)), evecs.T))
    
    return np.dot(X_centered, zca_matrix.T)

X_raw_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
X_sphered = perform_sphering(wells, CONTROL_NAME, REG_PARAM)

# ==========================================
# 5. UMAP & INTERACTIVE PLOTLY VISUALIZATION
# ==========================================
print("Running UMAP dimensionality reduction...")
n_neighbors = min(15, len(wells) - 1)
reducer = umap.UMAP(n_neighbors=n_neighbors, random_state=42)

emb_raw = reducer.fit_transform(X_raw_scaled)
emb_sph = reducer.fit_transform(X_sphered) if X_sphered is not None else None

# Build the interactive figure
fig = make_subplots(
    rows=1, cols=2, 
    subplot_titles=("Raw (Standardized)", f"Sphered (Centered on {CONTROL_NAME})")
)

def add_traces(fig, coords, col_idx):
    temp_df = wells[["Well_ID", "Treatment"]].copy()
    temp_df["x"] = coords[:, 0]
    temp_df["y"] = coords[:, 1]
    
    # Mutants: Black dots
    mutants = temp_df[temp_df["Treatment"] != CONTROL_NAME]
    fig.add_trace(go.Scatter(
        x=mutants["x"], y=mutants["y"],
        mode='markers', name='Mutant',
        marker=dict(color='black', size=7, opacity=0.6),
        text=[f"Mutant: {t}<br>Well: {w}" for t, w in zip(mutants["Treatment"], mutants["Well_ID"])],
        hoverinfo='text'
    ), row=1, col=col_idx)

    # Controls: Red X's
    controls = temp_df[temp_df["Treatment"] == CONTROL_NAME]
    fig.add_trace(go.Scatter(
        x=controls["x"], y=controls["y"],
        mode='markers', name='Control',
        marker=dict(color='red', size=10, symbol='x', line=dict(width=2)),
        text=[f"CONTROL: {t}<br>Well: {w}" for t, w in zip(controls["Treatment"], controls["Well_ID"])],
        hoverinfo='text'
    ), row=1, col=col_idx)

add_traces(fig, emb_raw, 1)
if emb_sph is not None:
    add_traces(fig, emb_sph, 2)

fig.update_layout(
    title="Interactive Mutant Analysis: Hover to Identify Wells",
    template="plotly_white",
    height=600, width=1200,
    showlegend=True
)

fig.show()

# Optional: Uncomment the line below to save this as a file you can open in a browser
# fig.write_html("Mutant_UMAP_Interactive.html")

In [ ]:
#zonder hierarcy, gwn mean direct
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ... (SETUP PATHS - SAME AS YOURS) ...
PROJECT_ROOT = r"E:\groteEdeepprofilerdingen\deepprofileroutputsetc\my_dp_project (Copy)"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")
CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
REG_PARAM = 1e-2

# ==========================================
# 2. WHOLE-WELL AGGREGATION (DIRECT)
# ==========================================
meta = pd.read_csv(METADATA_PATH)

# We use a dictionary to group cells by Well_ID before averaging
well_accumulator = {} 

print("Collecting all cells for whole-well aggregation...")
for i in tqdm(meta.index, desc="Reading Files"):
    well_id = f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}"
    treatment = str(meta.loc[i, TREATMENT_COL]).strip()
    
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")

    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                
                if len(cells_f) > 0:
                    if well_id not in well_accumulator:
                        well_accumulator[well_id] = {'features': [], 'treatment': treatment}
                    # Add all cells from this site to the well's list
                    well_accumulator[well_id]['features'].append(cells_f)
        except: continue

# Now, calculate the mean for each well across ALL its cells
well_level_data = []
for well_id, content in tqdm(well_accumulator.items(), desc="Calculating Well Means"):
    # Stack all sites' cells into one big array for this well
    all_cells_in_well = np.vstack(content['features'])
    well_mean = np.mean(all_cells_in_well, axis=0)
    
    well_dict = {"Well_ID": well_id, "Treatment": content['treatment']}
    # Add features as columns
    for idx, val in enumerate(well_mean):
        well_dict[idx] = val
    well_level_data.append(well_dict)

wells = pd.DataFrame(well_level_data)
feature_cols = [i for i in range(all_cells_in_well.shape[1])]

print(f"Done. Aggregated {len(wells)} wells directly from cell populations.")

# ==========================================
# 3. SPHERING & UMAP (SAME AS YOUR CODE)
# ==========================================
# ... [Insert your perform_sphering function here] ...

X_raw_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
X_sphered = perform_sphering(wells, CONTROL_NAME, REG_PARAM)

# ... [Insert your Plotly code here] ...

In [ ]:


# ==========================================
def perform_sphering(df, ctrl_label, reg):
    ctrl_wells = df[df["Treatment"] == ctrl_label][feature_cols].values
    if len(ctrl_wells) < 2:
        print("Warning: Need at least 2 control wells for sphering.")
        return None
    
    mean_vec = np.mean(ctrl_wells, axis=0)
    X_centered = df[feature_cols].values - mean_vec
    
    # Calculate ZCA Matrix
    cov = np.dot((ctrl_wells - mean_vec).T, (ctrl_wells - mean_vec)) / (len(ctrl_wells) - 1)
    evals, evecs = np.linalg.eigh(cov + reg * np.eye(cov.shape[0]))
    zca_matrix = np.dot(evecs, np.dot(np.diag(1.0 / np.sqrt(evals + 1e-6)), evecs.T))
    
    return np.dot(X_centered, zca_matrix.T)
X_sphered = perform_sphering(wells, CONTROL_NAME, REG_PARAM)

# 5. UMAP & INTERACTIVE PLOTLY VISUALIZATION
# ==========================================
print("Running UMAP dimensionality reduction...")
n_neighbors = min(15, len(wells) - 1)
reducer = umap.UMAP(n_neighbors=n_neighbors, random_state=42)

emb_raw = reducer.fit_transform(X_raw_scaled)
emb_sph = reducer.fit_transform(X_sphered) if X_sphered is not None else None

# Build the interactive figure
fig = make_subplots(
    rows=1, cols=2, 
    subplot_titles=("Raw (Standardized)", f"Sphered (Centered on {CONTROL_NAME})")
)

def add_traces(fig, coords, col_idx):
    temp_df = wells[["Well_ID", "Treatment"]].copy()
    temp_df["x"] = coords[:, 0]
    temp_df["y"] = coords[:, 1]
    
    # Mutants: Black dots
    mutants = temp_df[temp_df["Treatment"] != CONTROL_NAME]
    fig.add_trace(go.Scatter(
        x=mutants["x"], y=mutants["y"],
        mode='markers', name='Mutant',
        marker=dict(color='black', size=7, opacity=0.6),
        text=[f"Mutant: {t}<br>Well: {w}" for t, w in zip(mutants["Treatment"], mutants["Well_ID"])],
        hoverinfo='text'
    ), row=1, col=col_idx)

    # Controls: Red X's
    controls = temp_df[temp_df["Treatment"] == CONTROL_NAME]
    fig.add_trace(go.Scatter(
        x=controls["x"], y=controls["y"],
        mode='markers', name='Control',
        marker=dict(color='red', size=10, symbol='x', line=dict(width=2)),
        text=[f"CONTROL: {t}<br>Well: {w}" for t, w in zip(controls["Treatment"], controls["Well_ID"])],
        hoverinfo='text'
    ), row=1, col=col_idx)

add_traces(fig, emb_raw, 1)
if emb_sph is not None:
    add_traces(fig, emb_sph, 2)

fig.update_layout(
    title="Interactive Mutant Analysis: Hover to Identify Wells",
    template="plotly_white",
    height=600, width=1200,
    showlegend=True
)

fig.show()


In [ ]:
%pip install adjustText

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
#PROJECT_ROOT = r"E:\groteEdeepprofilerdingen\deepprofileroutputsetc\my_dp_project (Copy)"
PROJECT_ROOT = "/media/arnout/Elements/groteEdeepprofilerdingen/deepprofileroutputsetc/my_dp_project (Copy)"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(4, 3))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(12, 8))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# 1. SETUP PATHS
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/my_dp_project (Copy)"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# DeepProfiler Specs
NUM_CHANNELS = 5
FEATS_PER_CH = 1280 # 6400 / 5

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Voorbereiden van de DataFrame
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)

# Aggregatie naar Well-level
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION & MULTI-PLOT
# ==========================================
# Definieer de plot configuraties
plot_configs = [
    {"name": "All Channels Together", "indices": feature_cols},
    {"name": "Channel 1 Only", "indices": list(range(0, 1280))},
    {"name": "Channel 2 Only", "indices": list(range(1280, 2560))},
    {"name": "Channel 3 Only", "indices": list(range(2560, 3840))},
    {"name": "Channel 4 Only", "indices": list(range(3840, 5120))},
    {"name": "Channel 5 Only", "indices": list(range(5120, 6400))}
]

# Maak een grid van 2 rijen x 3 kolommen
fig, axes = plt.subplots(2, 3, figsize=(24, 16))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP: {config['name']}...")
    
    # Selectie en Schalen
    X_subset = wells[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # UMAP berekening
    reducer = umap.UMAP(n_neighbors=15, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    # Splitsen voor visualisatie
    is_control = wells["Treatment"] == CONTROL_NAME
    controls = embedding[is_control]
    mutants = embedding[~is_control]
    mutant_names = wells[~is_control]["Treatment"].values
    
    # Plotten van de punten
    ax.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=100, label='Control', zorder=3)
    ax.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=40, alpha=0.6, label='Mutants', zorder=2)
    
    # Labels toevoegen
    texts = []
    for i, name in enumerate(mutant_names):
        texts.append(ax.text(mutants[i, 0], mutants[i, 1], name, fontsize=8))
    
    # Adjust text zorgt dat labels niet overlappen per subplot
    adjust_text(texts, ax=ax, only_move={'points':'y', 'text':'xy'}, 
                arrowprops=dict(arrowstyle='->', color='gray', lw=0.5))
    
    ax.set_title(config['name'], fontsize=14, fontweight='bold')
    ax.set_facecolor('#fafafa')
    ax.grid(True, linestyle='--', alpha=0.3)
    if idx == 0:
        ax.legend(loc='upper left')

plt.suptitle(f"Channel-specific UMAP Comparison ({len(wells)} Wells)", fontsize=22, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# 1. SETUP PATHS
# ==========================================
#PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/my_dp_project (Copy)"
PROJECT_ROOT =r"E:\groteEdeepprofilerdingen\deepprofileroutputsetc\my_dp_project (Copy)"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# DeepProfiler Specs
NUM_CHANNELS = 5
FEATS_PER_CH = 1280 

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Prepare DataFrame
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)

# Aggregate to Well-level
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION & MULTI-PLOT
# ==========================================
plot_configs = [
    {"name": "All Channels Together", "indices": feature_cols},
    {"name": "Channel 1 Only", "indices": list(range(0, 1280))},
    {"name": "Channel 2 Only", "indices": list(range(1280, 2560))},
    {"name": "Channel 3 Only", "indices": list(range(2560, 3840))},
    {"name": "Channel 4 Only", "indices": list(range(3840, 5120))},
    {"name": "Channel 5 Only", "indices": list(range(5120, 6400))}
]

fig, axes = plt.subplots(2, 3, figsize=(24, 16))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP: {config['name']}...")
    
    # --- CRITICAL FIXES FOR CONSISTENCY ---
    # 1. Reset the global seed before EVERY UMAP run
    np.random.seed(42)
    
    # 2. Subset and Scale
    X_subset = wells[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # 3. UMAP with explicit initialization
    reducer = umap.UMAP(
        n_neighbors=15, 
        random_state=42, 
        init='spectral', # Stable initialization
        low_memory=False
    )
    embedding = reducer.fit_transform(X_scaled)
    # --------------------------------------

    is_control = wells["Treatment"] == CONTROL_NAME
    controls = embedding[is_control]
    mutants = embedding[~is_control]
    mutant_names = wells[~is_control]["Treatment"].values
    
    # Plotting
    ax.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=100, label='Control', zorder=3)
    ax.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=40, alpha=0.6, label='Mutants', zorder=2)
    
    texts = []
    for i, name in enumerate(mutant_names):
        texts.append(ax.text(mutants[i, 0], mutants[i, 1], name, fontsize=8))
    
    adjust_text(texts, ax=ax, only_move={'points':'y', 'text':'xy'}, 
                arrowprops=dict(arrowstyle='->', color='gray', lw=0.5))
    
    ax.set_title(config['name'], fontsize=14, fontweight='bold')
    ax.set_facecolor('#fafafa')
    ax.grid(True, linestyle='--', alpha=0.3)
    
    # Force the aspect ratio to be equal so the shapes aren't stretched
    ax.set_aspect('equal', 'datalim')

    if idx == 0:
        ax.legend(loc='upper left')

plt.suptitle(f"Channel-specific UMAP Comparison ({len(wells)} Wells)", fontsize=22, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# 1. SETUP PATHS
# ==========================================
PROJECT_ROOT = r"E:\groteEdeepprofilerdingen\deepprofileroutputsetc\my_dp_project (Copy)"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. DATA LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Aggregating sites to well-level...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)

wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP - TUNED FOR BETTER CLUSTERING
# ==========================================
X_subset = wells[feature_cols].values
X_scaled = StandardScaler().fit_transform(X_subset)

# We use 'random' init instead of 'spectral' because 'random' often 
# allows for the more "stretched" clusters you preferred in the 2nd code.
reducer = umap.UMAP(
    n_neighbors=15, 
    min_dist=0.1,    # Allows tighter clusters
    random_state=42, 
    init='random',   
    n_epochs=500     # More iterations for better convergence
)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING (Wide Aspect Ratio)
# ==========================================
# Making the plot wider (16x9) mimics the layout of the subplots you liked
plt.figure(figsize=(16, 9))
ax = plt.gca()

is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

texts = []
for i, name in enumerate(mutant_names):
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=10))

# adjust_text tuned for high-density labels
adjust_text(texts, 
            only_move={'points':'y', 'text':'xy'},
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5))

plt.title(f"Optimized UMAP: All Channels Together", fontsize=16, fontweight='bold')
plt.gca().set_facecolor('#fafafa')
plt.grid(True, linestyle='--', alpha=0.3)
plt.legend(loc='best')

plt.tight_layout()
plt.show()

In [ ]:
#channels kiezen
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# 1. SETUP & CONFIGURATION
# ==========================================
# Paden (Windows Raw String)
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/my_dp_project (Copy)"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# Kanaal configuratie
CHANNELS_TO_KEEP = [1]  # Verander dit naar de kanalen die je wilt zien
FEATS_PER_CHANNEL = 1280      # 6400 features / 5 kanalen

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

def get_channel_indices(channels, feats_per_ch):
    indices = []
    for ch in channels:
        start = (ch - 1) * feats_per_ch
        end = ch * feats_per_ch
        indices.extend(range(start, end))
    return indices

selected_indices = get_channel_indices(CHANNELS_TO_KEEP, FEATS_PER_CHANNEL)

print(f"Loading Sites and selecting features for channels {CHANNELS_TO_KEEP}...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                # Verwijder cellen met NaNs
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                
                if len(cells_f) > 0:
                    # Bereken median feature vector voor deze site
                    site_median = np.median(cells_f, axis=0)
                    
                    # Filter direct op de gewenste kanalen om geheugen te besparen
                    filtered_median = site_median[selected_indices]
                    
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(filtered_median)
        except:
            continue

# Maak DataFrames
feature_cols = [f"feat_{i}" for i in selected_indices]
sites_df = pd.concat([
    pd.DataFrame(site_level_data), 
    pd.DataFrame(site_level_features, columns=feature_cols)
], axis=1)

# Aggregeer naar Well-level (gemiddelde van de sites)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print(f"Running UMAP on {len(feature_cols)} features...")
X_raw = wells[feature_cols].values
X_scaled = StandardScaler().fit_transform(X_raw)

reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Splits data voor styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Teken de punten
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=150, 
            label=f'Control ({CONTROL_NAME})', zorder=3, linewidths=2)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=80, 
            alpha=0.6, label='Mutants', zorder=2)

# Voeg labels toe
texts = []
for i, name in enumerate(mutant_names):
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=10))

print("Adjusting labels for readability...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'xy'}, 
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.7, 1.7))

plt.title(f"Well-Level UMAP (Channels: {CHANNELS_TO_KEEP})", fontsize=16)
plt.xlabel("UMAP 1", fontsize=12)
plt.ylabel("UMAP 2", fontsize=12)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.gca().set_facecolor('#fafafa')
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#channels kiezen
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# 1. SETUP & CONFIGURATION
# ==========================================
# Paden (Windows Raw String)
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/my_dp_project (Copy)"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# Kanaal configuratie
CHANNELS_TO_KEEP = [1,2,4,5]  # Verander dit naar de kanalen die je wilt zien
FEATS_PER_CHANNEL = 1280      # 6400 features / 5 kanalen

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

def get_channel_indices(channels, feats_per_ch):
    indices = []
    for ch in channels:
        start = (ch - 1) * feats_per_ch
        end = ch * feats_per_ch
        indices.extend(range(start, end))
    return indices

selected_indices = get_channel_indices(CHANNELS_TO_KEEP, FEATS_PER_CHANNEL)

print(f"Loading Sites and selecting features for channels {CHANNELS_TO_KEEP}...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                # Verwijder cellen met NaNs
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                
                if len(cells_f) > 0:
                    # Bereken median feature vector voor deze site
                    site_median = np.median(cells_f, axis=0)
                    
                    # Filter direct op de gewenste kanalen om geheugen te besparen
                    filtered_median = site_median[selected_indices]
                    
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(filtered_median)
        except:
            continue

# Maak DataFrames
feature_cols = [f"feat_{i}" for i in selected_indices]
sites_df = pd.concat([
    pd.DataFrame(site_level_data), 
    pd.DataFrame(site_level_features, columns=feature_cols)
], axis=1)

# Aggregeer naar Well-level (gemiddelde van de sites)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print(f"Running UMAP on {len(feature_cols)} features...")
X_raw = wells[feature_cols].values
X_scaled = StandardScaler().fit_transform(X_raw)

reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Splits data voor styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Teken de punten
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=150, 
            label=f'Control ({CONTROL_NAME})', zorder=3, linewidths=2)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=80, 
            alpha=0.6, label='Mutants', zorder=2)

# Voeg labels toe
texts = []
for i, name in enumerate(mutant_names):
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=10))

print("Adjusting labels for readability...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'xy'}, 
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.7, 1.7))

plt.title(f"Well-Level UMAP (Channels: {CHANNELS_TO_KEEP})", fontsize=16)
plt.xlabel("UMAP 1", fontsize=12)
plt.ylabel("UMAP 2", fontsize=12)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.gca().set_facecolor('#fafafa')
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#channels kiezen
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# 1. SETUP & CONFIGURATION
# ==========================================
# Paden (Windows Raw String)
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/my_dp_project (Copy)"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# Kanaal configuratie
CHANNELS_TO_KEEP = [5]  # Verander dit naar de kanalen die je wilt zien
FEATS_PER_CHANNEL = 1280      # 6400 features / 5 kanalen

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

def get_channel_indices(channels, feats_per_ch):
    indices = []
    for ch in channels:
        start = (ch - 1) * feats_per_ch
        end = ch * feats_per_ch
        indices.extend(range(start, end))
    return indices

selected_indices = get_channel_indices(CHANNELS_TO_KEEP, FEATS_PER_CHANNEL)

print(f"Loading Sites and selecting features for channels {CHANNELS_TO_KEEP}...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                # Verwijder cellen met NaNs
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                
                if len(cells_f) > 0:
                    # Bereken median feature vector voor deze site
                    site_median = np.median(cells_f, axis=0)
                    
                    # Filter direct op de gewenste kanalen om geheugen te besparen
                    filtered_median = site_median[selected_indices]
                    
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(filtered_median)
        except:
            continue

# Maak DataFrames
feature_cols = [f"feat_{i}" for i in selected_indices]
sites_df = pd.concat([
    pd.DataFrame(site_level_data), 
    pd.DataFrame(site_level_features, columns=feature_cols)
], axis=1)

# Aggregeer naar Well-level (gemiddelde van de sites)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print(f"Running UMAP on {len(feature_cols)} features...")
X_raw = wells[feature_cols].values
X_scaled = StandardScaler().fit_transform(X_raw)

reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Splits data voor styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Teken de punten
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=150, 
            label=f'Control ({CONTROL_NAME})', zorder=3, linewidths=2)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=80, 
            alpha=0.6, label='Mutants', zorder=2)

# Voeg labels toe
texts = []
for i, name in enumerate(mutant_names):
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=10))

print("Adjusting labels for readability...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'xy'}, 
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.7, 1.7))

plt.title(f"Well-Level UMAP (Channels: {CHANNELS_TO_KEEP})", fontsize=16)
plt.xlabel("UMAP 1", fontsize=12)
plt.ylabel("UMAP 2", fontsize=12)
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.gca().set_facecolor('#fafafa')
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

# Replace with the path to one of your files
file_path = r"E:\groteEdeepprofilerdingen\deepprofileroutputsetc\my_dp_project (Copy)\outputs\results\features\PLATE1_T2\A1\1.npz"

# Load the file
with np.load(file_path) as data:
    # 1. See the 'keys' (the names of the arrays inside)
    print("Keys in this file:", data.files)
    
    # 2. Extract the features
    features = data['features']
    
    # 3. Check the dimensions
    # Shape will be (Number of Cells, Number of Features)
    print("Array Shape:", features.shape)
    
    # 4. Look at a small slice of the data (first 5 cells, first 5 features)
    print("Data Preview:\n", features[:5, :5])

In [ ]:
import pandas as pd
import numpy as np

file_path = r"E:\groteEdeepprofilerdingen\deepprofileroutputsetc\my_dp_project (Copy)\outputs\results\features\PLATE1_T2\A1\0.npz"

with np.load(file_path) as data:
    # Convert the numpy array to a Pandas DataFrame
    df = pd.DataFrame(data['features'])
    
    # Save to CSV
    df.to_csv("site_features_preview2.csv", index=False)
    print("Saved to site_features_preview.csv")

In [ ]:
#metmasking_length
import numpy as np

# Replace with the path to one of your files
file_path = "/media/arnout/Elements1/Thesis/project_masking_label/outputs/results/features/PLATE1_T2/A1/11.npz"

# Load the file
with np.load(file_path) as data:
    # 1. See the 'keys' (the names of the arrays inside)
    print("Keys in this file:", data.files)
    
    # 2. Extract the features
    features = data['features']
    
    # 3. Check the dimensions
    # Shape will be (Number of Cells, Number of Features)
    print("Array Shape:", features.shape)
    
    # 4. Look at a small slice of the data (first 5 cells, first 5 features)
    print("Data Preview:\n", features[:5, :5])

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

# ==========================================
# 1. SETUP PATHS & CONFIG
# ==========================================
# Update these to match your exact drive location
PROJECT_ROOT = "/media/arnout/Elements/Thesis/my_dp_project (Copy)/"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs/results/features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs/metadata/index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
REG_PARAM = 1e-2 # Regularization parameter for ZCA

# ==========================================
# 2. HIERARCHICAL DATA LOADING
# ==========================================
if not os.path.exists(METADATA_PATH):
    raise FileNotFoundError(f"Could not find index.csv at {METADATA_PATH}")

meta = pd.read_csv(METADATA_PATH)
site_level_data = []
site_level_features = []

print(f"Loading data from: {FEATURES_BASE}")
for i in tqdm(meta.index, desc="Sites Loaded"):
    # Path construction: Root/Plate/Well/Site.npz
    filename = os.path.join(
        FEATURES_BASE,
        str(meta.loc[i, "Metadata_Plate"]),
        str(meta.loc[i, "Metadata_Well"]),
        f"{meta.loc[i, 'Metadata_Site']}.npz"
    )

    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                # Filter out NaNs
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                
                if len(cells_f) > 0:
                    # AGGREGATION LEVEL 1: Site-level Median
                    site_median = np.median(cells_f, axis=0)
                    
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(site_median)
        except:
            continue

# Create Main DataFrame
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)

# ==========================================
# 3. AGGREGATION LEVEL 2: Well-level Mean
# ==========================================
# We keep wells individual so we can see the 'no_sgRNA' consistency check
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()
print(f"\nProcessing complete. Found {len(wells)} unique wells.")

# ==========================================
# 4. SPHERING (ZCA WHITENING) LOGIC
# ==========================================
def perform_sphering(df, ctrl_label, reg):
    # Extract control well features
    ctrl_wells = df[df["Treatment"] == ctrl_label][feature_cols].values
    if len(ctrl_wells) < 2:
        print(f"Warning: Not enough {ctrl_label} wells for covariance calculation.")
        return None
    
    # Center the entire dataset based on control mean
    mean_vec = np.mean(ctrl_wells, axis=0)
    X_centered = df[feature_cols].values - mean_vec
    
    # Calculate ZCA Matrix on control wells
    ctrl_centered = ctrl_wells - mean_vec
    cov = np.dot(ctrl_centered.T, ctrl_centered) / (len(ctrl_wells) - 1)
    evals, evecs = np.linalg.eigh(cov + reg * np.eye(cov.shape[0]))
    zca_matrix = np.dot(evecs, np.dot(np.diag(1.0 / np.sqrt(evals + 1e-6)), evecs.T))
    
    # Transform all data
    return np.dot(X_centered, zca_matrix.T)

# Prepare both datasets
X_raw_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
X_sphered = perform_sphering(wells, CONTROL_NAME, REG_PARAM)

# ==========================================
# 5. UMAP & VISUALIZATION
# ==========================================
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 10))
n_neighbors = min(15, len(wells) - 1)
reducer = umap.UMAP(n_neighbors=n_neighbors, random_state=42)

# Plot 1: Non-Sphered (Standardized Only)
print("Computing UMAP for Raw features...")
emb_raw = reducer.fit_transform(X_raw_scaled)
sns.scatterplot(x=emb_raw[:, 0], y=emb_raw[:, 1], hue=wells["Treatment"], 
                style=(wells["Treatment"] == CONTROL_NAME), markers={True: "X", False: "o"},
                s=200, ax=ax1, palette="turbo", alpha=0.8, edgecolor='w')
ax1.set_title("WELL-LEVEL: No Sphering (Standardized)\nChecking for technical plate effects", fontsize=15)

# Plot 2: Sphered
if X_sphered is not None:
    print("Computing UMAP for Sphered features...")
    emb_sph = reducer.fit_transform(X_sphered)
    sns.scatterplot(x=emb_sph[:, 0], y=emb_sph[:, 1], hue=wells["Treatment"], 
                    style=(wells["Treatment"] == CONTROL_NAME), markers={True: "X", False: "o"},
                    s=200, ax=ax2, palette="turbo", alpha=0.8, edgecolor='w')
    ax2.set_title(f"WELL-LEVEL: Sphered (Aligned to {CONTROL_NAME})\nControls (X) should cluster tightly", fontsize=15)

plt.tight_layout()
plt.show()

In [ ]:
###################testje#################################

In [ ]:
#exact zoals vorige plaat gerund: illuminatin and compression on false, maar toch gebeuren illuminaison and compression

import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#exact zoals vorige plaat gerund: illuminatin and compression on false, maar nu heb ik de illuminatin en compresison file andere naam gegeven: zelfde output dus ik denk dat bij vorige geen illum en compression is in rekening gebracht


import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#nu illumniaiton correction en compression: true, compresiso nen illumination files gnereeted EN nu worden ze ook gebruikt bij profiling


import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#wel illumination correction (true), geen compression (false)

import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#nog eens illumination corretction: ipv: calculate: true gwn deze zin wegdoen zoals in de voorbeeld files, geen compression


import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#geen illuminationcorr en geen compression (files echt gedeleted)


import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#masking


import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset_masking"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean

def get_aggregated_features(metadata_df, features_root):
    """
    Loads .npz files for each site, calculates the mean 
    (well-level or treatment-level) and returns a feature matrix.
    """
    all_features = []
    labels = []

    for index, row in metadata_df.iterrows():
        # Construct the path to the .npz file
        # DeepProfiler structure: outputs/features/<Plate>/<Well>_<Site>.npz
        plate = str(row['Metadata_Plate'])
        well = str(row['Metadata_Well'])
        site = str(row['Metadata_Site'])
        
        feat_path = os.path.join(features_root, plate, well, f"{site}.npz")
        
        if os.path.exists(feat_path):
            # Load .npz file
            data = np.load(feat_path)
            # 'features' is the standard key used by DeepProfiler
            feats = data['features'] 
            
            # Take the mean of all cells in this site to get the 'Site Centroid'
            site_mean = np.mean(feats, axis=0)
            
            all_features.append(site_mean)
            labels.append(row['Treatment'])
        else:
            print(f"Warning: Feature file not found: {feat_path}")

    return pd.DataFrame(all_features), labels

# 1. Load your main metadata (the index.csv you showed)
metadata = pd.read_csv("/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset/inputs/metadata/index.csv")

# 2. Set your output directories
# (Adjust these to where your profiling outputs are stored)
output_no_mask = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset/outputs/results /features"
output_with_mask = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset_masking/outputs/results/features"

# 3. Define the two mutants you want to compare
mutant_1 = "trnH" # Replace with your actual treatment name
mutant_2 = "dnaG" # Replace with your actual treatment name

def calculate_mutant_dist(feat_dir, meta_df, m1, m2):
    # Load and aggregate
    df_feats, labels = get_aggregated_features(meta_df, feat_dir)
    df_feats['Treatment'] = labels
    
    # Calculate Centroids for the specific mutants
    centroid1 = df_feats[df_feats['Treatment'] == m1].drop('Treatment', axis=1).mean()
    centroid2 = df_feats[df_feats['Treatment'] == m2].drop('Treatment', axis=1).mean()
    
    return euclidean(centroid1, centroid2)

# 4. Perform Calculation
print("Calculating distances...")
dist_no_mask = calculate_mutant_dist(output_no_mask, metadata, mutant_1, mutant_2)
dist_mask = calculate_mutant_dist(output_with_mask, metadata, mutant_1, mutant_2)

# 5. Output Results
print(f"\nResults for separation between {mutant_1} and {mutant_2}:")
print(f"--- Distance (No Mask):   {dist_no_mask:.4f}")
print(f"--- Distance (With Mask): {dist_mask:.4f}")

change = ((dist_mask - dist_no_mask) / dist_no_mask) * 100
print(f"--- Difference: {change:+.2f}%")

if change > 0:
    print("Interpretation: Masking increased the separation between these mutants.")
else:
    print("Interpretation: Masking reduced the separation (context might be important).")

In [ ]:

import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset_masking"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#alles_met_masking

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text  # Ensures labels don't overlap

# ==========================================
# 1. SETUP PATHS (Windows Raw String)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/my_dp_project_masking"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Flatten to Well-Level
num_features = site_level_features[0].shape[0]
feature_cols = [i for i in range(num_features)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP REDUCTION
# ==========================================
print("Running UMAP...")
X_scaled = StandardScaler().fit_transform(wells[feature_cols].values)
reducer = umap.UMAP(n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(X_scaled)

# ==========================================
# 4. PLOTTING WITH SMART LABELS
# ==========================================
plt.figure(figsize=(14, 10))

# Separate for styling
is_control = wells["Treatment"] == CONTROL_NAME
controls = embedding[is_control]
mutants = embedding[~is_control]
mutant_names = wells[~is_control]["Treatment"].values

# Plot dots
plt.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=3)
plt.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=60, alpha=0.6, label='Mutants', zorder=2)

# Create labels list
texts = []
for i, name in enumerate(mutant_names):
    # Only label if it's not the control
    texts.append(plt.text(mutants[i, 0], mutants[i, 1], name, fontsize=9, fontweight='medium'))

# Use adjust_text to repel labels from each other and the dots
print("Adjusting labels (this may take a few seconds)...")
adjust_text(texts, 
            only_move={'points':'y', 'text':'y'}, # Encourages horizontal reading
            arrowprops=dict(arrowstyle='->', color='gray', lw=0.5),
            expand_points=(1.5, 1.5))

plt.title(f"Well-Level UMAP: {len(wells)} Total Wells", fontsize=16)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(loc='best')
plt.gca().set_facecolor('#fafafa') # Slight grey background for contrast
plt.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# 1. SETUP PATHS
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/my_dp_project_masking"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Data voorbereiden
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()

# ==========================================
# 3. UMAP & PLOTTING LOOP
# ==========================================
# Definieer de subsets die we willen plotten
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

# Maak een grid van 2 rijen x 3 kolommen
fig, axes = plt.subplots(2, 3, figsize=(24, 16))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP for: {config['name']}...")
    
    # Selecteer data voor deze specifieke subset
    X_subset = wells[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # Run UMAP
    reducer = umap.UMAP(n_neighbors=15, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    # Data splitsen voor visualisatie
    is_control = wells["Treatment"] == CONTROL_NAME
    controls = embedding[is_control]
    mutants = embedding[~is_control]
    mutant_names = wells[~is_control]["Treatment"].values
    
    # Plotten
    ax.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=100, label='Control', zorder=3)
    ax.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=40, alpha=0.6, label='Mutants', zorder=2)
    
    # Labels toevoegen
    texts = []
    for i, name in enumerate(mutant_names):
        texts.append(ax.text(mutants[i, 0], mutants[i, 1], name, fontsize=8))
    
    adjust_text(texts, ax=ax, only_move={'points':'y', 'text':'xy'}, expand_points=(1.2, 1.2))
    
    ax.set_title(f"{config['name']}", fontsize=14, fontweight='bold')
    ax.set_facecolor('#fafafa')
    ax.grid(True, linestyle='--', alpha=0.3)
    if idx == 0:
        ax.legend(loc='upper left')

plt.suptitle(f"Channel Comparison UMAP - {len(wells)} Wells", fontsize=20, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
######aggregation and plotting seprate

In [ ]:
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# 1. SETUP PATHS
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/Thesis/my_dp_project_neighbours"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Data voorbereiden
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].mean().reset_index()


In [ ]:

# ==========================================
# 3. UMAP & PLOTTING LOOP
# ==========================================
# Definieer de subsets die we willen plotten
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

# Maak een grid van 2 rijen x 3 kolommen
fig, axes = plt.subplots(2, 3, figsize=(24, 16))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP for: {config['name']}...")
    
    # Selecteer data voor deze specifieke subset
    X_subset = wells[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # Run UMAP
    reducer = umap.UMAP(n_neighbors=15, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    # Data splitsen voor visualisatie
    is_control = wells["Treatment"] == CONTROL_NAME
    controls = embedding[is_control]
    mutants = embedding[~is_control]
    mutant_names = wells[~is_control]["Treatment"].values
    
    # Plotten
    ax.scatter(controls[:, 0], controls[:, 1], c='red', marker='x', s=100, label='Control', zorder=3)
    ax.scatter(mutants[:, 0], mutants[:, 1], c='black', marker='o', s=40, alpha=0.6, label='Mutants', zorder=2)
    
    # Labels toevoegen
    texts = []
    for i, name in enumerate(mutant_names):
        texts.append(ax.text(mutants[i, 0], mutants[i, 1], name, fontsize=8))
    
    adjust_text(texts, ax=ax, only_move={'points':'y', 'text':'xy'}, expand_points=(1.2, 1.2))
    
    ax.set_title(f"{config['name']}", fontsize=14, fontweight='bold')
    ax.set_facecolor('#fafafa')
    ax.grid(True, linestyle='--', alpha=0.3)
    if idx == 0:
        ax.legend(loc='upper left')

plt.suptitle(f"Channel Comparison UMAP - {len(wells)} Wells", fontsize=20, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#median:
import pandas as pd
import numpy as np
import os
import umap
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# 1. SETUP PATHS
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/Thesis/my_dp_project_neighbours"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280

# ==========================================
# 2. HIERARCHICAL LOADING & AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)
site_level_data, site_level_features = [], []

print("Loading and Aggregating Sites...")
for i in tqdm(meta.index):
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                if len(cells_f) > 0:
                    site_level_data.append({
                        "Well_ID": f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}",
                        "Treatment": str(meta.loc[i, TREATMENT_COL]).strip()
                    })
                    site_level_features.append(np.median(cells_f, axis=0))
        except: continue

# Data voorbereiden
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
sites_df = pd.concat([pd.DataFrame(site_level_data), 
                      pd.DataFrame(site_level_features, columns=feature_cols)], axis=1)
wells = sites_df.groupby(["Well_ID", "Treatment"])[feature_cols].median().reset_index()


In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# 3. UMAP & PLOTTING LOOP
# ==========================================
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

fig, axes = plt.subplots(2, 3, figsize=(24, 16))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP for: {config['name']}...")
    
    # Preprocessing
    X_subset = wells[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # Run UMAP
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    # Split for plotting
    is_control = wells["Treatment"] == CONTROL_NAME
    
    # Plotting
    ax.scatter(embedding[is_control, 0], embedding[is_control, 1], 
               c='red', marker='x', s=100, label='Control', zorder=3)
    ax.scatter(embedding[~is_control, 0], embedding[~is_control, 1], 
               c='black', marker='o', s=40, alpha=0.6, label='Mutants', zorder=2)
    
    # Labels
    mutant_names = wells[~is_control]["Treatment"].values
    mutant_embeds = embedding[~is_control]
    texts = [ax.text(mutant_embeds[i, 0], mutant_embeds[i, 1], name, fontsize=8) 
             for i, name in enumerate(mutant_names)]
    
    adjust_text(texts, ax=ax, only_move={'points':'y', 'text':'xy'}, expand_points=(1.2, 1.2))
    
    ax.set_title(f"{config['name']}", fontsize=14, fontweight='bold')
    ax.set_facecolor('#fafafa')
    ax.grid(True, linestyle='--', alpha=0.3)
    if idx == 0: ax.legend(loc='upper left')

plt.suptitle(f"Channel Comparison UMAP - {len(wells)} Wells", fontsize=20, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ==========================================
# 3. UMAP & PLOTTING (Full Standalone Cell)
# ==========================================
import umap
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# --- CONFIGURATION ---
# Ensure these match your aggregation cell variables
CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]

# Define the channel subsets
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

# Create Figure
fig, axes = plt.subplots(2, 3, figsize=(26, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP for: {config['name']}...")
    
    # 1. Prepare Data
    X_subset = wells[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # 2. Run UMAP
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    # 3. Split Data for Plotting
    is_control = wells[TREATMENT_COL] == CONTROL_NAME
    
    # Extract Control data
    ctrl_embed = embedding[is_control]
    ctrl_well_ids = wells[is_control]["Well_ID"].values
    
    # Extract Mutant data
    mutant_embed = embedding[~is_control]
    mutant_names = wells[~is_control][TREATMENT_COL].values
    
    # 4. Draw Scatter Points
    # Controls as Red Crosses
    ax.scatter(ctrl_embed[:, 0], ctrl_embed[:, 1], 
               c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=4)
    
    # Mutants as Black Dots
    ax.scatter(mutant_embed[:, 0], mutant_embed[:, 1], 
               c='black', marker='o', s=50, alpha=0.5, label='Mutants', zorder=3)
    
    # 5. Create Labels (Texts)
    texts = []
    
    # Add Red Well_ID labels for Controls
    for i, well_id in enumerate(ctrl_well_ids):
        texts.append(ax.text(ctrl_embed[i, 0], ctrl_embed[i, 1], 
                             well_id, fontsize=7, color='red', fontweight='bold'))
    
    # Add Black Treatment labels for Mutants
    for i, name in enumerate(mutant_names):
        texts.append(ax.text(mutant_embed[i, 0], mutant_embed[i, 1], 
                             name, fontsize=8, color='black'))
    
    # 6. Adjust Labels to prevent overlaps
    # arrowprops adds a subtle line from the text to the point if it moves
    adjust_text(texts, ax=ax, 
                arrowprops=dict(arrowstyle='-', color='silver', lw=0.5),
                only_move={'points':'y', 'text':'xy'}, 
                expand_points=(1.5, 1.5))
    
    # 7. Formatting
    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')
    ax.set_facecolor('#fafafa')
    ax.grid(True, linestyle='--', alpha=0.3)
    
    if idx == 0:
        ax.legend(loc='upper left', frameon=True, shadow=True)

# Final global formatting
plt.suptitle(f"Channel Comparison UMAP - {len(wells)} Wells (Double Median Aggregation)", 
             fontsize=22, y=1.02, fontweight='bold')
plt.tight_layout()

# Save plot (Optional)
# plt.savefig("umap_channel_comparison.png", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
#median over alles

import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. SETUP & PATHS
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/Thesis/my_dp_project_neighbours"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]

# ==========================================
# 2. SINGLE-STEP WELL AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)

# Dictionary to store all cells per well: { "Well_ID": [list_of_feature_arrays] }
well_storage = {}
# Dictionary to map Well_ID back to its Treatment name
well_to_treatment = {}

print("Loading cells and grouping by Well...")
for i in tqdm(meta.index):
    well_id = f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}"
    treatment = str(meta.loc[i, TREATMENT_COL]).strip()
    
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                # Clean NaNs
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                
                if len(cells_f) > 0:
                    if well_id not in well_storage:
                        well_storage[well_id] = []
                        well_to_treatment[well_id] = treatment
                    
                    well_storage[well_id].append(cells_f)
        except:
            continue

# Now calculate the median for each well using all pooled cells
well_level_data = []

print("Calculating Single Median per Well...")
for well_id, feature_list in tqdm(well_storage.items()):
    # Stack all cells from all sites into one large array for this well
    all_cells_in_well = np.vstack(feature_list)
    
    # Calculate median across the 0-axis (rows/cells)
    well_median = np.median(all_cells_in_well, axis=0)
    
    # Store result
    row = {"Well_ID": well_id, "Treatment": well_to_treatment[well_id]}
    for idx, val in enumerate(well_median):
        row[idx] = val
    well_level_data.append(row)

wells = pd.DataFrame(well_level_data)

print(f"Aggregation complete. Processed {len(wells)} wells.")

In [ ]:
# ==========================================
# 3. UMAP & PLOTTING (Full Standalone Cell)
# ==========================================
import umap
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# --- CONFIGURATION ---
# Ensure these match your aggregation cell variables
CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]

# Define the channel subsets
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))},
    
]

# Create Figure
fig, axes = plt.subplots(2, 3, figsize=(26, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP for: {config['name']}...")
    
    # 1. Prepare Data
    X_subset = wells[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # 2. Run UMAP
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    # 3. Split Data for Plotting
    is_control = wells[TREATMENT_COL] == CONTROL_NAME
    
    # Extract Control data
    ctrl_embed = embedding[is_control]
    ctrl_well_ids = wells[is_control]["Well_ID"].values
    
    # Extract Mutant data
    mutant_embed = embedding[~is_control]
    mutant_names = wells[~is_control][TREATMENT_COL].values
    
    # 4. Draw Scatter Points
    # Controls as Red Crosses
    ax.scatter(ctrl_embed[:, 0], ctrl_embed[:, 1], 
               c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=4)
    
    # Mutants as Black Dots
    ax.scatter(mutant_embed[:, 0], mutant_embed[:, 1], 
               c='black', marker='o', s=50, alpha=0.5, label='Mutants', zorder=3)
    
    # 5. Create Labels (Texts)
    texts = []
    
    # Add Red Well_ID labels for Controls
    for i, well_id in enumerate(ctrl_well_ids):
        texts.append(ax.text(ctrl_embed[i, 0], ctrl_embed[i, 1], 
                             well_id, fontsize=7, color='red', fontweight='bold'))
    
    # Add Black Treatment labels for Mutants
    for i, name in enumerate(mutant_names):
        texts.append(ax.text(mutant_embed[i, 0], mutant_embed[i, 1], 
                             name, fontsize=8, color='black'))
    
    # 6. Adjust Labels to prevent overlaps
    # arrowprops adds a subtle line from the text to the point if it moves
    adjust_text(texts, ax=ax, 
                arrowprops=dict(arrowstyle='-', color='silver', lw=0.5),
                only_move={'points':'y', 'text':'xy'}, 
                expand_points=(1.5, 1.5))
    
    # 7. Formatting
    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')
    ax.set_facecolor('#fafafa')
    ax.grid(True, linestyle='--', alpha=0.3)
    
    if idx == 0:
        ax.legend(loc='upper left', frameon=True, shadow=True)

# Final global formatting
plt.suptitle(f"Channel Comparison UMAP - {len(wells)} Wells (Double Median Aggregation)", 
             fontsize=22, y=1.02, fontweight='bold')
plt.tight_layout()

# Save plot (Optional)
# plt.savefig("umap_channel_comparison.png", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# ==========================================
# 3. UMAP & PLOTTING (With Channel Combinations)
# ==========================================
import umap
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# --- CONFIGURATION ---
CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
FEATS_PER_CH = 1280

# Helper function to combine channel indices
def get_ch_indices(channels):
    """
    channels: list of integers (e.g., [1, 2, 5])
    returns: flattened list of indices for those channels
    """
    indices = []
    for ch in channels:
        # Subtract 1 because list(range) is 0-indexed
        start = (ch - 1) * FEATS_PER_CH
        end = ch * FEATS_PER_CH
        indices.extend(list(range(start, end)))
    return indices

# Define your subsets here - You can add any combination you like!
plot_configs = [
    {"name": "All Channels", "indices": get_ch_indices([1, 2, 3, 4, 5])},
    {"name": "Channels 1, 2, 3, 5", "indices": get_ch_indices([1, 2, 3, 5])},
    {"name": "Channel 1 only", "indices": get_ch_indices([1])},
    {"name": "Channel 2 only", "indices": get_ch_indices([2])},
    {"name": "Channel 3 only", "indices": get_ch_indices([3])},
    {"name": "Channel 5 only", "indices": get_ch_indices([5])}
]

# Adjust grid size based on the number of configs
num_plots = len(plot_configs)
cols = 3
rows = (num_plots + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(26, 8 * rows))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP for: {config['name']}...")
    
    # 1. Prepare Data
    # Note: Use feature_cols if they are strings, but since they are integers from the aggregation cell:
    X_subset = wells[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # 2. Run UMAP
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    # 3. Split Data
    is_control = wells[TREATMENT_COL] == CONTROL_NAME
    ctrl_embed = embedding[is_control]
    ctrl_well_ids = wells[is_control]["Well_ID"].values
    mutant_embed = embedding[~is_control]
    mutant_names = wells[~is_control][TREATMENT_COL].values
    
    # 4. Draw Scatter Points
    ax.scatter(ctrl_embed[:, 0], ctrl_embed[:, 1], 
               c='red', marker='x', s=120, label='Control', zorder=4)
    ax.scatter(mutant_embed[:, 0], mutant_embed[:, 1], 
               c='black', marker='o', s=50, alpha=0.5, label='Mutants', zorder=3)
    
    # 5. Create Labels
    texts = []
    for i, well_id in enumerate(ctrl_well_ids):
        texts.append(ax.text(ctrl_embed[i, 0], ctrl_embed[i, 1], 
                             well_id, fontsize=7, color='red', fontweight='bold'))
    for i, name in enumerate(mutant_names):
        texts.append(ax.text(mutant_embed[i, 0], mutant_embed[i, 1], 
                             name, fontsize=8, color='black'))
    
    # 6. Adjust Labels
    adjust_text(texts, ax=ax, 
                arrowprops=dict(arrowstyle='-', color='silver', lw=0.5),
                only_move={'points':'y', 'text':'xy'}, 
                expand_points=(1.5, 1.5))
    
    # 7. Formatting
    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')
    ax.set_facecolor('#fafafa')
    ax.grid(True, linestyle='--', alpha=0.3)
    if idx == 0:
        ax.legend(loc='upper left')

# Hide unused subplots if plot_configs is not a multiple of 3
for j in range(idx + 1, len(axes)):
    axes[j].axis('off')

plt.suptitle(f"Channel Combination UMAP Comparison", fontsize=22, y=1.01, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#size overlay

In [ ]:
#median over alles

import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. SETUP & PATHS
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/Thesis/my_dp_project_neighbours"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]

# ==========================================
# 2. SINGLE-STEP WELL AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)

# Dictionary to store all cells per well: { "Well_ID": [list_of_feature_arrays] }
well_storage = {}
# Dictionary to map Well_ID back to its Treatment name
well_to_treatment = {}

print("Loading cells and grouping by Well...")
for i in tqdm(meta.index):
    well_id = f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}"
    treatment = str(meta.loc[i, TREATMENT_COL]).strip()
    
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                # Clean NaNs
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                
                if len(cells_f) > 0:
                    if well_id not in well_storage:
                        well_storage[well_id] = []
                        well_to_treatment[well_id] = treatment
                    
                    well_storage[well_id].append(cells_f)
        except:
            continue

# Now calculate the median for each well using all pooled cells
well_level_data = []

print("Calculating Single Median per Well...")
for well_id, feature_list in tqdm(well_storage.items()):
    # Stack all cells from all sites into one large array for this well
    all_cells_in_well = np.vstack(feature_list)
    
    # Calculate median across the 0-axis (rows/cells)
    well_median = np.median(all_cells_in_well, axis=0)
    
    # Store result
    row = {"Well_ID": well_id, "Treatment": well_to_treatment[well_id]}
    for idx, val in enumerate(well_median):
        row[idx] = val
    well_level_data.append(row)

wells = pd.DataFrame(well_level_data)

print(f"Aggregation complete. Processed {len(wells)} wells.")

# Load your area report
AREA_REPORT_PATH = "/media/arnout/Elements1/Thesis/my_dp_project_neighbours/inputs/cell_area_report.csv"
area_df = pd.read_csv(AREA_REPORT_PATH)

# Create the same Well_ID used in your features (Plate_Well)
area_df["Well_ID"] = area_df["Plate"] + "_" + area_df["Well"]

# Calculate the average area per well
well_areas = area_df.groupby("Well_ID")["Area"].median().reset_index()

# Merge this area info into your 'wells' feature dataframe
wells = wells.merge(well_areas, on="Well_ID", how="left")

In [ ]:
#size color overlay
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ... [Keep your config and plot_configs as they were] ...
# ==========================================
# 3. UMAP & PLOTTING (Full Standalone Cell)
# ==========================================
import umap
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# --- CONFIGURATION ---
# Ensure these match your aggregation cell variables
CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]

# Define the channel subsets
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))},
    
]

fig, axes = plt.subplots(2, 3, figsize=(26, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP for: {config['name']}...")
    
    X_subset = wells[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    # Split Data
    is_control = wells[TREATMENT_COL] == CONTROL_NAME
    
    # 1. Controls (Red Crosses - Area ignored for color to keep them distinct)
    ctrl_embed = embedding[is_control]
    ax.scatter(ctrl_embed[:, 0], ctrl_embed[:, 1], 
               c='red', marker='x', s=150, label='Control (no_sgRNA)', zorder=5)
    
    # 2. Mutants (Colored by Area)
    mutant_embed = embedding[~is_control]
    mutant_areas = wells[~is_control]["Area"].values
    mutant_names = wells[~is_control][TREATMENT_COL].values

    # We use a colormap like 'viridis' or 'plasma'
    # 'vmax' and 'vmin' can be set to clip outliers if needed
    sc = ax.scatter(mutant_embed[:, 0], mutant_embed[:, 1], 
                    c=mutant_areas, cmap='viridis', marker='o', 
                    s=80, alpha=0.8, edgecolors='white', lw=0.5, zorder=4)
    
    # 3. Add Labels
    texts = []
    # Controls
    for i, well_id in enumerate(wells[is_control]["Well_ID"].values):
        texts.append(ax.text(ctrl_embed[i, 0], ctrl_embed[i, 1], well_id, color='red', fontsize=8))
    # Mutants
    for i, name in enumerate(mutant_names):
        texts.append(ax.text(mutant_embed[i, 0], mutant_embed[i, 1], name, fontsize=8))

    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='silver', lw=0.5))
    
    # 4. Colorbar (Only add once or to each subplot)
    cbar = plt.colorbar(sc, ax=ax, shrink=0.6)
    cbar.set_label('Median Cell Area (Pixels)', fontsize=10)

    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')

plt.suptitle(f"UMAP Colored by Cell Area - {len(wells)} Wells", fontsize=22, y=1.02, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#also individual cells overlay (to be done)

import umap
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# ==========================================
# CONFIGURATION FOR OVERLAY
# ==========================================
# List mutants (by Treatment name) and controls (by Well_ID) to show individual cells
CELL_OVERLAY_MUTANTS = ["glmM", "glmS"] 
CELL_OVERLAY_CONTROLS = ["PLATE1_T2_E11"] 

# Colors for the specific overlays
overlay_colors = ['cyan', 'lime', 'magenta', 'orange', 'yellow']

# ==========================================
# UMAP & PLOTTING
# ==========================================
fig, axes = plt.subplots(2, 3, figsize=(26, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP for: {config['name']}...")
    
    # 1. Prepare Well-Level Data
    X_wells = wells[config['indices']].values
    scaler = StandardScaler()
    X_wells_scaled = scaler.fit_transform(X_wells)
    
    # 2. Fit UMAP on Wells
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_wells_scaled)
    
    # 3. Plot Background (All Wells)
    is_control = wells[TREATMENT_COL] == CONTROL_NAME
    ax.scatter(embedding[is_control, 0], embedding[is_control, 1], 
               c='red', marker='x', s=100, label='Control Wells', zorder=2, alpha=0.5)
    ax.scatter(embedding[~is_control, 0], embedding[~is_control, 1], 
               c='black', marker='o', s=40, label='Mutant Wells', zorder=1, alpha=0.3)

    # 4. OVERLAY INDIVIDUAL CELLS
    color_idx = 0
    
    # Process Mutants in the list
    for mutant in CELL_OVERLAY_MUTANTS:
        # Find all wells matching this treatment
        matching_wells = wells[wells[TREATMENT_COL] == mutant]["Well_ID"].values
        all_cells = []
        for w_id in matching_wells:
            if w_id in well_storage:
                all_cells.append(np.vstack(well_storage[w_id]))
        
        if all_cells:
            # Get features for specific channels, scale them, and transform through UMAP
            X_cells = np.vstack(all_cells)[:, config['indices']]
            X_cells_scaled = scaler.transform(X_cells) # Use well-level scaler!
            cell_embed = reducer.transform(X_cells_scaled) # Transform into well-space
            
            c = overlay_colors[color_idx % len(overlay_colors)]
            ax.scatter(cell_embed[:, 0], cell_embed[:, 1], c=c, s=5, alpha=0.2, 
                       label=f"Cells: {mutant}", zorder=3)
            color_idx += 1

    # Process Controls in the list
    for ctrl_well in CELL_OVERLAY_CONTROLS:
        if ctrl_well in well_storage:
            X_cells = np.vstack(well_storage[ctrl_well])[:, config['indices']]
            X_cells_scaled = scaler.transform(X_cells)
            cell_embed = reducer.transform(X_cells_scaled)
            
            c = overlay_colors[color_idx % len(overlay_colors)]
            ax.scatter(cell_embed[:, 0], cell_embed[:, 1], c=c, s=5, alpha=0.2, 
                       label=f"Cells: {ctrl_well}", zorder=3, marker='.')
            color_idx += 1

    # 5. Add Labels for Wells (Existing logic)
    texts = []
    # Only label the wells we are highlighting to keep it clean, or keep your old logic:
    for i, row in wells.iterrows():
        if row[TREATMENT_COL] in CELL_OVERLAY_MUTANTS or row["Well_ID"] in CELL_OVERLAY_CONTROLS:
            texts.append(ax.text(embedding[i, 0], embedding[i, 1], 
                                 row["Well_ID"] if row[TREATMENT_COL] == CONTROL_NAME else row[TREATMENT_COL],
                                 fontsize=9, fontweight='bold'))

    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='->', color='black', lw=1))
    
    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')
    if idx == 0:
        ax.legend(loc='upper left', markerscale=2)

plt.suptitle("UMAP Well Medians with Individual Cell Overlay", fontsize=22, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#masking met label, size threhsold 700, neibhours 4

In [ ]:
#median over alles

import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. SETUP & PATHS
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements1/Thesis/project_masking_label"
FEATURES_BASE = os.path.join(PROJECT_ROOT, "outputs", "results", "features")
METADATA_PATH = os.path.join(PROJECT_ROOT, "inputs", "metadata", "index.csv")

CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]

# ==========================================
# 2. SINGLE-STEP WELL AGGREGATION
# ==========================================
meta = pd.read_csv(METADATA_PATH)

# Dictionary to store all cells per well: { "Well_ID": [list_of_feature_arrays] }
well_storage = {}
# Dictionary to map Well_ID back to its Treatment name
well_to_treatment = {}

print("Loading cells and grouping by Well...")
for i in tqdm(meta.index):
    well_id = f"{meta.loc[i, 'Metadata_Plate']}_{meta.loc[i, 'Metadata_Well']}"
    treatment = str(meta.loc[i, TREATMENT_COL]).strip()
    
    filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Plate"]), 
                            str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
    
    if os.path.isfile(filename):
        try:
            with np.load(filename) as data:
                cells = data["features"]
                # Clean NaNs
                cells_f = cells[~np.isnan(cells).any(axis=1)]
                
                if len(cells_f) > 0:
                    if well_id not in well_storage:
                        well_storage[well_id] = []
                        well_to_treatment[well_id] = treatment
                    
                    well_storage[well_id].append(cells_f)
        except:
            continue

# Now calculate the median for each well using all pooled cells
well_level_data = []

print("Calculating Single Median per Well...")
for well_id, feature_list in tqdm(well_storage.items()):
    # Stack all cells from all sites into one large array for this well
    all_cells_in_well = np.vstack(feature_list)
    
    # Calculate median across the 0-axis (rows/cells)
    well_median = np.median(all_cells_in_well, axis=0)
    
    # Store result
    row = {"Well_ID": well_id, "Treatment": well_to_treatment[well_id]}
    for idx, val in enumerate(well_median):
        row[idx] = val
    well_level_data.append(row)

wells = pd.DataFrame(well_level_data)

print(f"Aggregation complete. Processed {len(wells)} wells.")

In [ ]:
# ==========================================
# 3. UMAP & PLOTTING (Full Standalone Cell)
# ==========================================
import umap
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# --- CONFIGURATION ---
# Ensure these match your aggregation cell variables
CONTROL_NAME = "no_sgRNA" 
TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]

# Define the channel subsets
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

# Create Figure
fig, axes = plt.subplots(2, 3, figsize=(26, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Processing UMAP for: {config['name']}...")
    
    # 1. Prepare Data
    X_subset = wells[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # 2. Run UMAP
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    # 3. Split Data for Plotting
    is_control = wells[TREATMENT_COL] == CONTROL_NAME
    
    # Extract Control data
    ctrl_embed = embedding[is_control]
    ctrl_well_ids = wells[is_control]["Well_ID"].values
    
    # Extract Mutant data
    mutant_embed = embedding[~is_control]
    mutant_names = wells[~is_control][TREATMENT_COL].values
    
    # 4. Draw Scatter Points
    # Controls as Red Crosses
    ax.scatter(ctrl_embed[:, 0], ctrl_embed[:, 1], 
               c='red', marker='x', s=120, label='Control (no_sgRNA)', zorder=4)
    
    # Mutants as Black Dots
    ax.scatter(mutant_embed[:, 0], mutant_embed[:, 1], 
               c='black', marker='o', s=50, alpha=0.5, label='Mutants', zorder=3)
    
    # 5. Create Labels (Texts)
    texts = []
    
    # Add Red Well_ID labels for Controls
    for i, well_id in enumerate(ctrl_well_ids):
        texts.append(ax.text(ctrl_embed[i, 0], ctrl_embed[i, 1], 
                             well_id, fontsize=7, color='red', fontweight='bold'))
    
    # Add Black Treatment labels for Mutants
    for i, name in enumerate(mutant_names):
        texts.append(ax.text(mutant_embed[i, 0], mutant_embed[i, 1], 
                             name, fontsize=8, color='black'))
    
    # 6. Adjust Labels to prevent overlaps
    # arrowprops adds a subtle line from the text to the point if it moves
    adjust_text(texts, ax=ax, 
                arrowprops=dict(arrowstyle='-', color='silver', lw=0.5),
                only_move={'points':'y', 'text':'xy'}, 
                expand_points=(1.5, 1.5))
    
    # 7. Formatting
    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')
    ax.set_facecolor('#fafafa')
    ax.grid(True, linestyle='--', alpha=0.3)
    
    if idx == 0:
        ax.legend(loc='upper left', frameon=True, shadow=True)

# Final global formatting
plt.suptitle(f"Channel Comparison UMAP - {len(wells)} Wells (Double Median Aggregation)", 
             fontsize=22, y=1.02, fontweight='bold')
plt.tight_layout()

# Save plot (Optional)
# plt.savefig("umap_channel_comparison.png", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
#T1_T2

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. SETUP & PATHS (MULTI-PLATE)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements/groteEdeepprofilerdingen/DataDeepprofiler"
PLATES = ["PLATE1_T0","PLATE1_T1","PLATE1_T2"] # List your folder names here

TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280

all_plates_data = []

for plate_id in PLATES:
    print(f"\n--- Processing {plate_id} ---")
    
    # Define paths specific to this plate
    FEATURES_BASE = os.path.join(PROJECT_ROOT,"features", plate_id)
    METADATA_PATH = os.path.join(PROJECT_ROOT,"metadata", f"index_{plate_id}.csv")
    
    meta = pd.read_csv(METADATA_PATH)
    well_storage = {}
    well_to_treatment = {}

    # Load cells per well
    for i in tqdm(meta.index, desc=f"Loading {plate_id}"):
        well_id = f"{plate_id}_{meta.loc[i, 'Metadata_Well']}"
        treatment = str(meta.loc[i, TREATMENT_COL]).strip()
        
        # Adjust filename path logic to match your folder structure
        filename = os.path.join(FEATURES_BASE, 
                                str(meta.loc[i, "Metadata_Well"]), 
                                f"{meta.loc[i, 'Metadata_Site']}.npz")
        
        if os.path.isfile(filename):
            try:
                with np.load(filename) as data:
                    cells = data["features"]
                    cells_f = cells[~np.isnan(cells).any(axis=1)]
                    
                    if len(cells_f) > 0:
                        if well_id not in well_storage:
                            well_storage[well_id] = []
                            well_to_treatment[well_id] = treatment
                        well_storage[well_id].append(cells_f)
            except:
                continue

    # Calculate median per well for THIS plate
    for well_id, feature_list in well_storage.items():
        all_cells_in_well = np.vstack(feature_list)
        well_median = np.median(all_cells_in_well, axis=0)
        
        row = {"Plate": plate_id, "Well_ID": well_id, "Treatment": well_to_treatment[well_id]}
        for idx, val in enumerate(well_median):
            row[idx] = val
        all_plates_data.append(row)

# Combine everything into one giant DataFrame
wells_df = pd.DataFrame(all_plates_data)
print(f"\nAggregation complete. Total Wells: {len(wells_df)}")

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# Define the channel subsets (same as before)
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

# LOOP THROUGH EACH PLATE INDIVIDUALLY
for plate_id in wells_df['Plate'].unique():
    plate_subset = wells_df[wells_df['Plate'] == plate_id].copy()
    
    fig, axes = plt.subplots(2, 3, figsize=(26, 18))
    axes = axes.flatten()

    for idx, config in enumerate(plot_configs):
        ax = axes[idx]
        
        # 1. Prepare & Scale Data
        X_subset = plate_subset[config['indices']].values
        X_scaled = StandardScaler().fit_transform(X_subset)
        
        # 2. Run UMAP
        reducer = umap.UMAP(n_neighbors=min(15, len(plate_subset)-1), min_dist=0.1, random_state=42)
        embedding = reducer.fit_transform(X_scaled)
        
        # 3. Plotting logic (same as your original code)
        is_control = plate_subset[TREATMENT_COL] == "no_sgRNA"
        
        # Draw Control vs Mutants
        ax.scatter(embedding[is_control, 0], embedding[is_control, 1], 
                   c='red', marker='x', s=120, label='Control', zorder=4)
        ax.scatter(embedding[~is_control, 0], embedding[~is_control, 1], 
                   c='black', marker='o', s=50, alpha=0.5, label='Mutants', zorder=3)

        # 4. Text Labels (using subset data)
        texts = []
        for i, row in enumerate(plate_subset.itertuples()):
            color = 'red' if row.Treatment == "no_sgRNA" else 'black'
            label = row.Well_ID if row.Treatment == "no_sgRNA" else row.Treatment
            texts.append(ax.text(embedding[i, 0], embedding[i, 1], label, fontsize=7, color=color))

        adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='silver', lw=0.5))
        ax.set_title(f"{plate_id} - {config['name']}")

    plt.suptitle(f"UMAP Analysis: {plate_id}", fontsize=22, y=1.02)
    plt.tight_layout()
    # plt.savefig(f"umap_{plate_id}.png")
    plt.show()

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text
import matplotlib.cm as cm

# 1. Setup Colors for Time Points (Plates)
# You can use a specific color map or define a manual dictionary
plate_list = sorted(wells_df['Plate'].unique())  # Ensures T0, T1, T2 order
colors = ['#66c2a5', '#fc8d62', '#8da0cb'] # Qualitative colors (or use a gradient)
plate_color_map = {plate: colors[i] for i, plate in enumerate(plate_list)}

feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

fig, axes = plt.subplots(2, 3, figsize=(28, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Running UMAP for {config['name']}...")
    
    # 1. Scale all data together
    X_subset = wells_df[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # 2. Run UMAP (on the whole dataset)
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    texts = []
    
    # 3. Plot Plate by Plate
    for plate in plate_list:
        mask = wells_df['Plate'] == plate
        plate_embed = embedding[mask]
        plate_meta = wells_df[mask]
        
        # Separate Control and Mutants within this plate
        is_ctrl = plate_meta[TREATMENT_COL] == "no_sgRNA"
        
        # Plot Mutants (Dots)
        ax.scatter(plate_embed[~is_ctrl, 0], plate_embed[~is_ctrl, 1], 
                   c=plate_color_map[plate], marker='o', s=60, alpha=0.6, 
                   label=f"{plate} (Mutant)")
        
        # Plot Controls (Crosses - kept red to stand out, or use plate color)
        ax.scatter(plate_embed[is_ctrl, 0], plate_embed[is_ctrl, 1], 
                   edgecolors=plate_color_map[plate], facecolors='none', marker='s', s=130, 
                   linewidths=2, label=f"{plate} (Control)")

        # 4. Add labels for this plate
        for i, row in enumerate(plate_meta.itertuples()):
            # Only label Mutants to avoid clutter, or label everything
            label = f"{row.Treatment}_{row.Plate.split('_')[-1]}" # e.g. "MutantA_T0"
            texts.append(ax.text(plate_embed[i, 0], plate_embed[i, 1], label, 
                                 fontsize=6, color=plate_color_map[plate]))

    # Formatting
    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')
    if idx == 0:
        ax.legend(loc='best', markerscale=1, fontsize=10, ncol=2)
    
    # Prevent overlap
    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='gray', alpha=0.3))

plt.suptitle("Combined UMAP: Time-Series Progression (T0 -> T1 -> T2)", fontsize=24, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
####with sphering
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. SETUP & PATHS
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements/groteEdeepprofilerdingen/DataDeepprofiler"
PLATES = ["PLATE1_T0", "PLATE1_T1", "PLATE1_T2"]

TREATMENT_COL = "Treatment"
CONTROL_NAME = "no_sgRNA"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]

# ==========================================
# 2. HELPER FUNCTION: SPHERING (WHITENING)
# ==========================================
def apply_sphering(df, feat_cols, control_val="no_sgRNA", reg_param=0.01):
    """
    Applies ZCA-Whitening to a dataframe based on its control samples.
    """
    X = df[feat_cols].values
    control_mask = df[TREATMENT_COL] == control_val
    X_control = X[control_mask]
    
    if len(X_control) < 2:
        print(f"Warning: Not enough control wells to sphere. Skipping whitening.")
        return df

    # Center based on control mean
    mu_control = np.mean(X_control, axis=0)
    X_centered = X - mu_control
    X_ctrl_centered = X_control - mu_control
    
    # Covariance & SVD
    sigma = np.dot(X_ctrl_centered.T, X_ctrl_centered) / (X_ctrl_centered.shape[0] - 1)
    # Adding regularization (REG_PARAM) to the diagonal for stability
    sigma = sigma + reg_param * np.eye(sigma.shape[0])
    
    eigenvalues, eigenvectors = np.linalg.eigh(sigma)
    
    # ZCA Whitening matrix
    # W = U * diag(1/sqrt(lambda)) * U.T
    W = np.dot(eigenvectors, np.dot(np.diag(1.0 / np.sqrt(eigenvalues + 1e-6)), eigenvectors.T))
    
    X_white = np.dot(X_centered, W)
    
    df_white = df.copy()
    df_white[feat_cols] = X_white
    return df_white

# ==========================================
# 3. AGGREGATION LOOP
# ==========================================
all_plates_list = []

for plate_id in PLATES:
    print(f"\n--- Aggregating {plate_id} ---")
    FEATURES_BASE = os.path.join(PROJECT_ROOT, "features", plate_id)
    METADATA_PATH = os.path.join(PROJECT_ROOT, "metadata", f"index_{plate_id}.csv")
    
    meta = pd.read_csv(METADATA_PATH)
    well_storage = {}
    well_to_treatment = {}

    for i in tqdm(meta.index, desc=f"Loading cells"):
        well_id = f"{plate_id}_{meta.loc[i, 'Metadata_Well']}"
        treatment = str(meta.loc[i, TREATMENT_COL]).strip()
        filename = os.path.join(FEATURES_BASE, str(meta.loc[i, "Metadata_Well"]), f"{meta.loc[i, 'Metadata_Site']}.npz")
        
        if os.path.isfile(filename):
            try:
                with np.load(filename) as data:
                    cells = data["features"]
                    cells_f = cells[~np.isnan(cells).any(axis=1)]
                    if len(cells_f) > 0:
                        if well_id not in well_storage:
                            well_storage[well_id] = []
                            well_to_treatment[well_id] = treatment
                        well_storage[well_id].append(cells_f)
            except:
                continue

    # Calculate Medians for the current plate
    current_plate_rows = []
    for well_id, feature_list in well_storage.items():
        all_cells_in_well = np.vstack(feature_list)
        well_median = np.median(all_cells_in_well, axis=0)
        row = {"Plate": plate_id, "Well_ID": well_id, "Treatment": well_to_treatment[well_id]}
        for idx, val in enumerate(well_median):
            row[idx] = val
        current_plate_rows.append(row)
    
    # Convert plate to DF and immediately sphere it
    plate_df = pd.DataFrame(current_plate_rows)
    print(f"Applying Sphering to {plate_id}...")
    plate_df_white = apply_sphering(plate_df, feature_cols, control_val=CONTROL_NAME)
    
    all_plates_list.append(plate_df_white)

# Final combined and whitened dataframe
wells_df = pd.concat(all_plates_list, ignore_index=True)
print(f"\nAggregation & Sphering complete. Total Wells: {len(wells_df)}")

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# Plate Color Mapping
plate_list = sorted(wells_df['Plate'].unique())
colors = ['#1f77b4', '#ff7f0e', '#2ca02c'] # Blue, Orange, Green for T0, T1, T2
plate_color_map = {plate: colors[i] for i, plate in enumerate(plate_list)}

plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

fig, axes = plt.subplots(2, 3, figsize=(26, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"UMAP for {config['name']}...")
    
    # Scale across all plates (standardizing the whitened ranges)
    X_subset = wells_df[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    texts = []
    
    for plate in plate_list:
        mask = wells_df['Plate'] == plate
        plate_embed = embedding[mask]
        plate_meta = wells_df[mask]
        
        is_ctrl = plate_meta[TREATMENT_COL] == CONTROL_NAME
        p_color = plate_color_map[plate]
        
        # Plot Mutants
        ax.scatter(plate_embed[~is_ctrl, 0], plate_embed[~is_ctrl, 1], 
                   c=p_color, marker='o', s=70, alpha=0.6, label=f"{plate}")
        
        # Plot Controls (Red X's for all plates to see if they overlap in center)
        ax.scatter(plate_embed[is_ctrl, 0], plate_embed[is_ctrl, 1], 
                   c='red', marker='x', s=100, alpha=0.8)

        # Labels
        for i, row in enumerate(plate_meta.itertuples()):
            if row.Treatment != CONTROL_NAME:
                label = f"{row.Treatment}_{row.Plate[-2:]}" # e.g. MutantA_T0
                texts.append(ax.text(plate_embed[i, 0], plate_embed[i, 1], label, 
                                     fontsize=7, color=p_color))

    ax.set_title(f"{config['name']}", fontweight='bold')
    if idx == 0:
        ax.legend(title="Plates", loc='best')

    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='silver', lw=0.5))

plt.suptitle("Whitened Multi-Plate UMAP (By Channel Subset)", fontsize=22, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#PLATE1 en 2 T1 en T2

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. SETUP & PATHS (MULTI-PLATE)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements/groteEdeepprofilerdingen/DataDeepprofiler"
PLATES = ["PLATE1_T0","PLATE1_T1","PLATE1_T2","PLATE2_T0","PLATE2_T1","PLATE2_T2"] # List your folder names here

TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280

all_plates_data = []

for plate_id in PLATES:
    print(f"\n--- Processing {plate_id} ---")
    
    # Define paths specific to this plate
    FEATURES_BASE = os.path.join(PROJECT_ROOT,"features", plate_id)
    METADATA_PATH = os.path.join(PROJECT_ROOT,"metadata", f"index_{plate_id}.csv")
    
    meta = pd.read_csv(METADATA_PATH)
    well_storage = {}
    well_to_treatment = {}

    # Load cells per well
    for i in tqdm(meta.index, desc=f"Loading {plate_id}"):
        well_id = f"{plate_id}_{meta.loc[i, 'Metadata_Well']}"
        treatment = str(meta.loc[i, TREATMENT_COL]).strip()
        
        # Adjust filename path logic to match your folder structure
        filename = os.path.join(FEATURES_BASE, 
                                str(meta.loc[i, "Metadata_Well"]), 
                                f"{meta.loc[i, 'Metadata_Site']}.npz")
        
        if os.path.isfile(filename):
            try:
                with np.load(filename) as data:
                    cells = data["features"]
                    cells_f = cells[~np.isnan(cells).any(axis=1)]
                    
                    if len(cells_f) > 0:
                        if well_id not in well_storage:
                            well_storage[well_id] = []
                            well_to_treatment[well_id] = treatment
                        well_storage[well_id].append(cells_f)
            except:
                continue

    # Calculate median per well for THIS plate
    for well_id, feature_list in well_storage.items():
        all_cells_in_well = np.vstack(feature_list)
        well_median = np.median(all_cells_in_well, axis=0)
        
        row = {"Plate": plate_id, "Well_ID": well_id, "Treatment": well_to_treatment[well_id]}
        for idx, val in enumerate(well_median):
            row[idx] = val
        all_plates_data.append(row)

# Combine everything into one giant DataFrame
wells_df = pd.DataFrame(all_plates_data)
print(f"\nAggregation complete. Total Wells: {len(wells_df)}")

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# Define the channel subsets (same as before)
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

# LOOP THROUGH EACH PLATE INDIVIDUALLY
for plate_id in wells_df['Plate'].unique():
    plate_subset = wells_df[wells_df['Plate'] == plate_id].copy()
    
    fig, axes = plt.subplots(2, 3, figsize=(26, 18))
    axes = axes.flatten()

    for idx, config in enumerate(plot_configs):
        ax = axes[idx]
        
        # 1. Prepare & Scale Data
        X_subset = plate_subset[config['indices']].values
        X_scaled = StandardScaler().fit_transform(X_subset)
        
        # 2. Run UMAP
        reducer = umap.UMAP(n_neighbors=min(15, len(plate_subset)-1), min_dist=0.1, random_state=42)
        embedding = reducer.fit_transform(X_scaled)
        
        # 3. Plotting logic (same as your original code)
        is_control = plate_subset[TREATMENT_COL] == "no_sgRNA"
        
        # Draw Control vs Mutants
        ax.scatter(embedding[is_control, 0], embedding[is_control, 1], 
                   c='red', marker='x', s=120, label='Control', zorder=4)
        ax.scatter(embedding[~is_control, 0], embedding[~is_control, 1], 
                   c='black', marker='o', s=50, alpha=0.5, label='Mutants', zorder=3)

        # 4. Text Labels (using subset data)
        texts = []
        for i, row in enumerate(plate_subset.itertuples()):
            color = 'red' if row.Treatment == "no_sgRNA" else 'black'
            label = row.Well_ID if row.Treatment == "no_sgRNA" else row.Treatment
            texts.append(ax.text(embedding[i, 0], embedding[i, 1], label, fontsize=7, color=color))

        adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='silver', lw=0.5))
        ax.set_title(f"{plate_id} - {config['name']}")

    plt.suptitle(f"UMAP Analysis: {plate_id}", fontsize=22, y=1.02)
    plt.tight_layout()
    # plt.savefig(f"umap_{plate_id}.png")
    plt.show()

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text
import matplotlib.cm as cm
import numpy as np

# --- 1. Dynamic Color Setup ---
plate_list = sorted(wells_df['Plate'].unique())
num_plates = len(plate_list)

# Use a qualitative colormap (Set1, Set3, or Dark2 are good for distinct categories)
# 'tab10' or 'tab20' are excellent for up to 10 or 20 distinct categories
cmap = plt.get_cmap('tab10') 
plate_colors = [cmap(i) for i in np.linspace(0, 1, num_plates)]
plate_color_map = {plate: plate_colors[i] for i, plate in enumerate(plate_list)}

# --- 2. Configuration ---
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

fig, axes = plt.subplots(2, 3, figsize=(28, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Running UMAP for {config['name']}...")
    
    X_subset = wells_df[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    texts = []
    
    # 3. Plot Plate by Plate
    for plate in plate_list:
        mask = wells_df['Plate'] == plate
        plate_embed = embedding[mask]
        plate_meta = wells_df[mask]
        
        is_ctrl = plate_meta[TREATMENT_COL] == "no_sgRNA"
        
        # Plot Mutants (Filled Circles)
        ax.scatter(plate_embed[~is_ctrl, 0], plate_embed[~is_ctrl, 1], 
                   color=plate_color_map[plate], marker='o', s=60, alpha=0.6, 
                   label=f"{plate} (Mutant)")
        
        # Plot Controls (Open Squares)
        ax.scatter(plate_embed[is_ctrl, 0], plate_embed[is_ctrl, 1], 
                   edgecolors=plate_color_map[plate], facecolors='none', 
                   marker='s', s=130, linewidths=2, label=f"{plate} (Control)")

        # 4. Add labels
        for i, row in enumerate(plate_meta.itertuples()):
            label = f"{row.Treatment}_{row.Plate.split('_')[-1]}"
            texts.append(ax.text(plate_embed[i, 0], plate_embed[i, 1], label, 
                                 fontsize=6, color=plate_color_map[plate]))

    # Formatting
    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')
    
    # Move legend to the side if it gets too crowded with 6+ plates
    if idx == 0:
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), markerscale=1, fontsize=8, ncol=1)
    
    # Adjust text to prevent overlapping labels
    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='gray', alpha=0.3))

plt.suptitle("Combined UMAP: Multi-Plate Analysis", fontsize=24, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#tsne and pca

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from adjustText import adjust_text

# --- 1. Configuration & Setup ---
# Choose your method here: "PCA" or "TSNE"
REDUCTION_METHOD = "PCA" 

plate_list = sorted(wells_df['Plate'].unique())
num_plates = len(plate_list)
cmap = plt.get_cmap('tab10')
plate_color_map = {plate: cmap(i % 10) for i, plate in enumerate(plate_list)}

feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

fig, axes = plt.subplots(2, 3, figsize=(28, 18))
axes = axes.flatten()

# --- 2. Iterative Plotting ---
for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Running {REDUCTION_METHOD} for {config['name']}...")
    
    X_subset = wells_df[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # Switch logic for Dimensionality Reduction
    if REDUCTION_METHOD == "PCA":
        reducer = PCA(n_components=2, random_state=42)
        embedding = reducer.fit_transform(X_scaled)
        expl_var = reducer.explained_variance_ratio_
        xlabel, ylabel = f"PC1 ({expl_var[0]:.1%})", f"PC2 ({expl_var[1]:.1%})"
    else:
        # t-SNE: Perplexity is key! (usually 5-50)
        reducer = TSNE(n_components=2, perplexity=min(30, len(wells_df)-1), 
                       init='pca', learning_rate='auto', random_state=42)
        embedding = reducer.fit_transform(X_scaled)
        xlabel, ylabel = "t-SNE 1", "t-SNE 2"
    
    texts = []
    
    for plate in plate_list:
        mask = wells_df['Plate'] == plate
        plate_embed = embedding[mask]
        plate_meta = wells_df[mask]
        is_ctrl = plate_meta[TREATMENT_COL] == "no_sgRNA"
        
        # Mutants
        ax.scatter(plate_embed[~is_ctrl, 0], plate_embed[~is_ctrl, 1], 
                   color=plate_color_map[plate], marker='o', s=70, alpha=0.7, 
                   label=f"{plate} (Mutant)")
        
        # Controls
        ax.scatter(plate_embed[is_ctrl, 0], plate_embed[is_ctrl, 1], 
                   edgecolors=plate_color_map[plate], facecolors='none', 
                   marker='s', s=150, linewidths=2.5, label=f"{plate} (Control)")

        for i, row in enumerate(plate_meta.itertuples()):
            label = f"{row.Treatment}_{row.Plate.split('_')[-1]}"
            texts.append(ax.text(plate_embed[i, 0], plate_embed[i, 1], label, 
                                 fontsize=7, color='black', alpha=0.8))

    # Formatting
    ax.set_title(f"{config['name']} ({REDUCTION_METHOD})", fontsize=16, fontweight='bold')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
    if idx == 0:
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=9)
    
    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='gray', alpha=0.2))

plt.suptitle(f"Combined {REDUCTION_METHOD}: Multi-Plate Analysis", fontsize=26, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from adjustText import adjust_text

# --- 1. Configuration & Setup ---
# Choose your method here: "PCA" or "TSNE"
REDUCTION_METHOD = "TSNE" 

plate_list = sorted(wells_df['Plate'].unique())
num_plates = len(plate_list)
cmap = plt.get_cmap('tab10')
plate_color_map = {plate: cmap(i % 10) for i, plate in enumerate(plate_list)}

feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

fig, axes = plt.subplots(2, 3, figsize=(28, 18))
axes = axes.flatten()

# --- 2. Iterative Plotting ---
for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Running {REDUCTION_METHOD} for {config['name']}...")
    
    X_subset = wells_df[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    # Switch logic for Dimensionality Reduction
    if REDUCTION_METHOD == "PCA":
        reducer = PCA(n_components=2, random_state=42)
        embedding = reducer.fit_transform(X_scaled)
        expl_var = reducer.explained_variance_ratio_
        xlabel, ylabel = f"PC1 ({expl_var[0]:.1%})", f"PC2 ({expl_var[1]:.1%})"
    else:
        # t-SNE: Perplexity is key! (usually 5-50)
        reducer = TSNE(n_components=2, perplexity=min(30, len(wells_df)-1), 
                       init='pca', learning_rate='auto', random_state=42)
        embedding = reducer.fit_transform(X_scaled)
        xlabel, ylabel = "t-SNE 1", "t-SNE 2"
    
    texts = []
    
    for plate in plate_list:
        mask = wells_df['Plate'] == plate
        plate_embed = embedding[mask]
        plate_meta = wells_df[mask]
        is_ctrl = plate_meta[TREATMENT_COL] == "no_sgRNA"
        
        # Mutants
        ax.scatter(plate_embed[~is_ctrl, 0], plate_embed[~is_ctrl, 1], 
                   color=plate_color_map[plate], marker='o', s=70, alpha=0.7, 
                   label=f"{plate} (Mutant)")
        
        # Controls
        ax.scatter(plate_embed[is_ctrl, 0], plate_embed[is_ctrl, 1], 
                   edgecolors=plate_color_map[plate], facecolors='none', 
                   marker='s', s=150, linewidths=2.5, label=f"{plate} (Control)")

        for i, row in enumerate(plate_meta.itertuples()):
            label = f"{row.Treatment}_{row.Plate.split('_')[-1]}"
            texts.append(ax.text(plate_embed[i, 0], plate_embed[i, 1], label, 
                                 fontsize=7, color='black', alpha=0.8))

    # Formatting
    ax.set_title(f"{config['name']} ({REDUCTION_METHOD})", fontsize=16, fontweight='bold')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
    if idx == 0:
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=9)
    
    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='gray', alpha=0.2))

plt.suptitle(f"Combined {REDUCTION_METHOD}: Multi-Plate Analysis", fontsize=26, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. SETUP & PATHS (MULTI-PLATE)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements/groteEdeepprofilerdingen/DataDeepprofiler"
PLATES = ["PLATE1_T2","PLATE2_T2_deduplication"] # List your folder names here

TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280

all_plates_data = []

for plate_id in PLATES:
    print(f"\n--- Processing {plate_id} ---")
    
    # Define paths specific to this plate
    FEATURES_BASE = os.path.join(PROJECT_ROOT,"features", plate_id)
    METADATA_PATH = os.path.join(PROJECT_ROOT,"metadata", f"index_{plate_id}.csv")
    
    meta = pd.read_csv(METADATA_PATH)
    well_storage = {}
    well_to_treatment = {}

    # Load cells per well
    for i in tqdm(meta.index, desc=f"Loading {plate_id}"):
        well_id = f"{plate_id}_{meta.loc[i, 'Metadata_Well']}"
        treatment = str(meta.loc[i, TREATMENT_COL]).strip()
        
        # Adjust filename path logic to match your folder structure
        filename = os.path.join(FEATURES_BASE, 
                                str(meta.loc[i, "Metadata_Well"]), 
                                f"{meta.loc[i, 'Metadata_Site']}.npz")
        
        if os.path.isfile(filename):
            try:
                with np.load(filename) as data:
                    cells = data["features"]
                    cells_f = cells[~np.isnan(cells).any(axis=1)]
                    
                    if len(cells_f) > 0:
                        if well_id not in well_storage:
                            well_storage[well_id] = []
                            well_to_treatment[well_id] = treatment
                        well_storage[well_id].append(cells_f)
            except:
                continue

    # Calculate median per well for THIS plate
    for well_id, feature_list in well_storage.items():
        all_cells_in_well = np.vstack(feature_list)
        well_median = np.median(all_cells_in_well, axis=0)
        
        row = {"Plate": plate_id, "Well_ID": well_id, "Treatment": well_to_treatment[well_id]}
        for idx, val in enumerate(well_median):
            row[idx] = val
        all_plates_data.append(row)

# Combine everything into one giant DataFrame
wells_df = pd.DataFrame(all_plates_data)
print(f"\nAggregation complete. Total Wells: {len(wells_df)}")

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text
import matplotlib.cm as cm
import numpy as np

# --- 1. Dynamic Color Setup ---
plate_list = sorted(wells_df['Plate'].unique())
num_plates = len(plate_list)

# Use a qualitative colormap (Set1, Set3, or Dark2 are good for distinct categories)
# 'tab10' or 'tab20' are excellent for up to 10 or 20 distinct categories
cmap = plt.get_cmap('tab10') 
plate_colors = [cmap(i) for i in np.linspace(0, 1, num_plates)]
plate_color_map = {plate: plate_colors[i] for i, plate in enumerate(plate_list)}

# --- 2. Configuration ---
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

fig, axes = plt.subplots(2, 3, figsize=(28, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Running UMAP for {config['name']}...")
    
    X_subset = wells_df[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    texts = []
    
    # 3. Plot Plate by Plate
    for plate in plate_list:
        mask = wells_df['Plate'] == plate
        plate_embed = embedding[mask]
        plate_meta = wells_df[mask]
        
        is_ctrl = plate_meta[TREATMENT_COL] == "no_sgRNA"
        
        # Plot Mutants (Filled Circles)
        ax.scatter(plate_embed[~is_ctrl, 0], plate_embed[~is_ctrl, 1], 
                   color=plate_color_map[plate], marker='o', s=60, alpha=0.6, 
                   label=f"{plate} (Mutant)")
        
        # Plot Controls (Open Squares)
        ax.scatter(plate_embed[is_ctrl, 0], plate_embed[is_ctrl, 1], 
                   edgecolors=plate_color_map[plate], facecolors='none', 
                   marker='s', s=130, linewidths=2, label=f"{plate} (Control)")

        # 4. Add labels
        for i, row in enumerate(plate_meta.itertuples()):
            label = f"{row.Treatment}_{row.Plate.split('_')[-1]}"
            texts.append(ax.text(plate_embed[i, 0], plate_embed[i, 1], label, 
                                 fontsize=6, color=plate_color_map[plate]))

    # Formatting
    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')
    
    # Move legend to the side if it gets too crowded with 6+ plates
    if idx == 0:
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), markerscale=1, fontsize=8, ncol=1)
    
    # Adjust text to prevent overlapping labels
    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='gray', alpha=0.3))

plt.suptitle("Combined UMAP: Multi-Plate Analysis", fontsize=24, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
#deduplication

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. SETUP & PATHS (MULTI-PLATE)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements/groteEdeepprofilerdingen/DataDeepprofiler"
PLATES = ["PLATE2_T0_deduplication","PLATE2_T1_deduplication","PLATE2_T2_deduplication"] # List your folder names here

TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280

all_plates_data = []

for plate_id in PLATES:
    print(f"\n--- Processing {plate_id} ---")
    
    # Define paths specific to this plate
    FEATURES_BASE = os.path.join(PROJECT_ROOT,"features", plate_id)
    METADATA_PATH = os.path.join(PROJECT_ROOT,"metadata", f"index_{plate_id}.csv")
    
    meta = pd.read_csv(METADATA_PATH)
    well_storage = {}
    well_to_treatment = {}

    # Load cells per well
    for i in tqdm(meta.index, desc=f"Loading {plate_id}"):
        well_id = f"{plate_id}_{meta.loc[i, 'Metadata_Well']}"
        treatment = str(meta.loc[i, TREATMENT_COL]).strip()
        
        # Adjust filename path logic to match your folder structure
        filename = os.path.join(FEATURES_BASE, 
                                str(meta.loc[i, "Metadata_Well"]), 
                                f"{meta.loc[i, 'Metadata_Site']}.npz")
        
        if os.path.isfile(filename):
            try:
                with np.load(filename) as data:
                    cells = data["features"]
                    cells_f = cells[~np.isnan(cells).any(axis=1)]
                    
                    if len(cells_f) > 0:
                        if well_id not in well_storage:
                            well_storage[well_id] = []
                            well_to_treatment[well_id] = treatment
                        well_storage[well_id].append(cells_f)
            except:
                continue

    # Calculate median per well for THIS plate
    for well_id, feature_list in well_storage.items():
        all_cells_in_well = np.vstack(feature_list)
        well_median = np.median(all_cells_in_well, axis=0)
        
        row = {"Plate": plate_id, "Well_ID": well_id, "Treatment": well_to_treatment[well_id]}
        for idx, val in enumerate(well_median):
            row[idx] = val
        all_plates_data.append(row)

# Combine everything into one giant DataFrame
wells_df = pd.DataFrame(all_plates_data)
print(f"\nAggregation complete. Total Wells: {len(wells_df)}")

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# Define the channel subsets (same as before)
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

# LOOP THROUGH EACH PLATE INDIVIDUALLY
for plate_id in wells_df['Plate'].unique():
    plate_subset = wells_df[wells_df['Plate'] == plate_id].copy()
    
    fig, axes = plt.subplots(2, 3, figsize=(26, 18))
    axes = axes.flatten()

    for idx, config in enumerate(plot_configs):
        ax = axes[idx]
        
        # 1. Prepare & Scale Data
        X_subset = plate_subset[config['indices']].values
        X_scaled = StandardScaler().fit_transform(X_subset)
        
        # 2. Run UMAP
        reducer = umap.UMAP(n_neighbors=min(15, len(plate_subset)-1), min_dist=0.1, random_state=42)
        embedding = reducer.fit_transform(X_scaled)
        
        # 3. Plotting logic (same as your original code)
        is_control = plate_subset[TREATMENT_COL] == "no_sgRNA"
        
        # Draw Control vs Mutants
        ax.scatter(embedding[is_control, 0], embedding[is_control, 1], 
                   c='red', marker='x', s=120, label='Control', zorder=4)
        ax.scatter(embedding[~is_control, 0], embedding[~is_control, 1], 
                   c='black', marker='o', s=50, alpha=0.5, label='Mutants', zorder=3)

        # 4. Text Labels (using subset data)
        texts = []
        for i, row in enumerate(plate_subset.itertuples()):
            color = 'red' if row.Treatment == "no_sgRNA" else 'black'
            label = row.Well_ID if row.Treatment == "no_sgRNA" else row.Treatment
            texts.append(ax.text(embedding[i, 0], embedding[i, 1], label, fontsize=7, color=color))

        adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='silver', lw=0.5))
        ax.set_title(f"{plate_id} - {config['name']}")

    plt.suptitle(f"UMAP Analysis: {plate_id}", fontsize=22, y=1.02)
    plt.tight_layout()
    # plt.savefig(f"umap_{plate_id}.png")
    plt.show()

In [ ]:
#mean

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. SETUP & PATHS (MULTI-PLATE)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements/groteEdeepprofilerdingen/DataDeepprofiler"
PLATES = ["PLATE2_T0_deduplication", "PLATE2_T1_deduplication", "PLATE2_T2_deduplication"]

TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280
TOTAL_FEATURES = NUM_CHANNELS * FEATS_PER_CH

all_plates_rows = []

for plate_id in PLATES:
    print(f"\n--- Processing {plate_id} ---")
    
    # Define paths specific to this plate
    FEATURES_BASE = os.path.join(PROJECT_ROOT, "features", plate_id)
    METADATA_PATH = os.path.join(PROJECT_ROOT, "metadata", f"index_{plate_id}.csv")
    
    if not os.path.exists(METADATA_PATH):
        print(f"Warning: Metadata not found for {plate_id}. Skipping.")
        continue

    meta = pd.read_csv(METADATA_PATH)
    well_storage = {}
    well_to_treatment = {}

    # Load cells per well
    for i in tqdm(meta.index, desc=f"Loading {plate_id}"):
        well_name = str(meta.loc[i, 'Metadata_Well'])
        well_id = f"{plate_id}_{well_name}"
        treatment = str(meta.loc[i, TREATMENT_COL]).strip()
        
        # Adjust filename path logic
        filename = os.path.join(FEATURES_BASE, 
                                well_name, 
                                f"{meta.loc[i, 'Metadata_Site']}.npz")
        
        if os.path.isfile(filename):
            try:
                with np.load(filename) as data:
                    cells = data["features"]
                    # Filter out NaNs
                    cells_f = cells[~np.isnan(cells).any(axis=1)]
                    
                    if len(cells_f) > 0:
                        if well_id not in well_storage:
                            well_storage[well_id] = []
                            well_to_treatment[well_id] = treatment
                        well_storage[well_id].append(cells_f)
            except Exception as e:
                continue

    # Calculate MEAN per well for THIS plate
    print(f"Aggregating {len(well_storage)} wells...")
    for well_id, feature_list in well_storage.items():
        # Stack all sites for this well into one array
        all_cells_in_well = np.vstack(feature_list)
        
        # SWITCHED TO MEAN: Faster and captures the average signal
        well_mean = np.mean(all_cells_in_well, axis=0)
        
        # Create a single row: Metadata + 6400 feature values
        row_data = [plate_id, well_id, well_to_treatment[well_id]] + well_mean.tolist()
        all_plates_rows.append(row_data)

# ==========================================
# 2. CREATE FINAL DATAFRAME
# ==========================================
# Generate column names: Metadata first, then feature indices
feature_cols = [f"feat_{i}" for i in range(TOTAL_FEATURES)]
columns = ["Plate", "Well_ID", "Treatment"] + feature_cols

wells_df = pd.DataFrame(all_plates_rows, columns=columns)

print(f"\n--- SUCCESS ---")
print(f"Aggregation complete. Total Wells: {len(wells_df)}")
print(f"Dataframe Shape: {wells_df.shape}")

# Optional: Display the first few rows
# print(wells_df.head())

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# 1. Define the channel subsets using the "feat_N" naming convention
# This matches the column names created in the previous step
plot_configs = [
    {"name": "All Channels", "indices": [f"feat_{i}" for i in range(TOTAL_FEATURES)]},
    {"name": "Channel 1", "indices": [f"feat_{i}" for i in range(0, 1280)]},
    {"name": "Channel 2", "indices": [f"feat_{i}" for i in range(1280, 2560)]},
    {"name": "Channel 3", "indices": [f"feat_{i}" for i in range(2560, 3840)]},
    {"name": "Channel 4", "indices": [f"feat_{i}" for i in range(3840, 5120)]},
    {"name": "Channel 5", "indices": [f"feat_{i}" for i in range(5120, 6400)]}
]

# 2. LOOP THROUGH EACH PLATE INDIVIDUALLY
for plate_id in wells_df['Plate'].unique():
    plate_subset = wells_df[wells_df['Plate'] == plate_id].copy().reset_index(drop=True)
    
    # Check if we have enough data to plot
    if len(plate_subset) < 2:
        print(f"Skipping {plate_id}: Not enough wells for UMAP.")
        continue

    fig, axes = plt.subplots(2, 3, figsize=(26, 18))
    axes = axes.flatten()

    for idx, config in enumerate(plot_configs):
        ax = axes[idx]
        
        # 1. Prepare & Scale Data
        # Accessing by the string names we created (feat_0, feat_1...)
        X_subset = plate_subset[config['indices']].values
        X_scaled = StandardScaler().fit_transform(X_subset)
        
        # 2. Run UMAP
        # n_neighbors cannot be larger than the number of samples
        n_neigh = min(15, len(plate_subset) - 1)
        reducer = umap.UMAP(n_neighbors=n_neigh, min_dist=0.1, random_state=42)
        embedding = reducer.fit_transform(X_scaled)
        
        # 3. Plotting logic
        # Filter for control group labels
        is_control = plate_subset['Treatment'] == "no_sgRNA"
        
        # Draw Control vs Mutants
        ax.scatter(embedding[is_control, 0], embedding[is_control, 1], 
                   c='red', marker='x', s=120, label='Control', zorder=4)
        ax.scatter(embedding[~is_control, 0], embedding[~is_control, 1], 
                   c='black', marker='o', s=50, alpha=0.5, label='Mutants', zorder=3)

        # 4. Text Labels
        texts = []
        for i, row in enumerate(plate_subset.itertuples()):
            color = 'red' if row.Treatment == "no_sgRNA" else 'black'
            # Label with Well_ID if control, otherwise use Treatment name
            label = row.Well_ID if row.Treatment == "no_sgRNA" else row.Treatment
            texts.append(ax.text(embedding[i, 0], embedding[i, 1], label, fontsize=8, color=color))

        # Avoid overlapping labels
        adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='silver', lw=0.5))
        ax.set_title(f"{config['name']}", fontsize=14)
        ax.grid(True, linestyle='--', alpha=0.3)

    plt.suptitle(f"UMAP Analysis (Mean Aggregation): {plate_id}", fontsize=22, y=1.02)
    plt.tight_layout()
    # plt.savefig(f"umap_mean_{plate_id}.png", bbox_inches='tight')
    plt.show()

In [ ]:
#masking comparison

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# ==========================================
# 1. SETUP & PATHS (MULTI-PLATE)
# ==========================================
PROJECT_ROOT = "/media/arnout/Elements/groteEdeepprofilerdingen/DataDeepprofiler"
PLATES = ["PLATE2_T0","PLATE2_T1","PLATE2_T2", "PLATE2_T0_masking","PLATE2_T1_masking","PLATE2_T2_masking"] # List your folder names here

TREATMENT_COL = "Treatment"
NUM_CHANNELS = 5
FEATS_PER_CH = 1280

all_plates_data = []

for plate_id in PLATES:
    print(f"\n--- Processing {plate_id} ---")
    
    # Define paths specific to this plate
    FEATURES_BASE = os.path.join(PROJECT_ROOT,"features", plate_id)
    METADATA_PATH = os.path.join(PROJECT_ROOT,"metadata", f"index_{plate_id}.csv")
    
    meta = pd.read_csv(METADATA_PATH)
    well_storage = {}
    well_to_treatment = {}

    # Load cells per well
    for i in tqdm(meta.index, desc=f"Loading {plate_id}"):
        well_id = f"{plate_id}_{meta.loc[i, 'Metadata_Well']}"
        treatment = str(meta.loc[i, TREATMENT_COL]).strip()
        
        # Adjust filename path logic to match your folder structure
        filename = os.path.join(FEATURES_BASE, 
                                str(meta.loc[i, "Metadata_Well"]), 
                                f"{meta.loc[i, 'Metadata_Site']}.npz")
        
        if os.path.isfile(filename):
            try:
                with np.load(filename) as data:
                    cells = data["features"]
                    cells_f = cells[~np.isnan(cells).any(axis=1)]
                    
                    if len(cells_f) > 0:
                        if well_id not in well_storage:
                            well_storage[well_id] = []
                            well_to_treatment[well_id] = treatment
                        well_storage[well_id].append(cells_f)
            except:
                continue

    # Calculate median per well for THIS plate
    for well_id, feature_list in well_storage.items():
        all_cells_in_well = np.vstack(feature_list)
        well_median = np.median(all_cells_in_well, axis=0)
        
        row = {"Plate": plate_id, "Well_ID": well_id, "Treatment": well_to_treatment[well_id]}
        for idx, val in enumerate(well_median):
            row[idx] = val
        all_plates_data.append(row)

# Combine everything into one giant DataFrame
wells_df = pd.DataFrame(all_plates_data)
print(f"\nAggregation complete. Total Wells: {len(wells_df)}")

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text

# Define the channel subsets (same as before)
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

# LOOP THROUGH EACH PLATE INDIVIDUALLY
for plate_id in wells_df['Plate'].unique():
    plate_subset = wells_df[wells_df['Plate'] == plate_id].copy()
    
    fig, axes = plt.subplots(2, 3, figsize=(26, 18))
    axes = axes.flatten()

    for idx, config in enumerate(plot_configs):
        ax = axes[idx]
        
        # 1. Prepare & Scale Data
        X_subset = plate_subset[config['indices']].values
        X_scaled = StandardScaler().fit_transform(X_subset)
        
        # 2. Run UMAP
        reducer = umap.UMAP(n_neighbors=min(15, len(plate_subset)-1), min_dist=0.1, random_state=42)
        embedding = reducer.fit_transform(X_scaled)
        
        # 3. Plotting logic (same as your original code)
        is_control = plate_subset[TREATMENT_COL] == "no_sgRNA"
        
        # Draw Control vs Mutants
        ax.scatter(embedding[is_control, 0], embedding[is_control, 1], 
                   c='red', marker='x', s=120, label='Control', zorder=4)
        ax.scatter(embedding[~is_control, 0], embedding[~is_control, 1], 
                   c='black', marker='o', s=50, alpha=0.5, label='Mutants', zorder=3)

        # 4. Text Labels (using subset data)
        texts = []
        for i, row in enumerate(plate_subset.itertuples()):
            color = 'red' if row.Treatment == "no_sgRNA" else 'black'
            label = row.Well_ID if row.Treatment == "no_sgRNA" else row.Treatment
            texts.append(ax.text(embedding[i, 0], embedding[i, 1], label, fontsize=7, color=color))

        adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='silver', lw=0.5))
        ax.set_title(f"{plate_id} - {config['name']}")

    plt.suptitle(f"UMAP Analysis: {plate_id}", fontsize=22, y=1.02)
    plt.tight_layout()
    # plt.savefig(f"umap_{plate_id}.png")
    plt.show()

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text
import matplotlib.cm as cm
import numpy as np

# --- 1. Dynamic Color Setup ---
plate_list = sorted(wells_df['Plate'].unique())
num_plates = len(plate_list)

# Use a qualitative colormap (Set1, Set3, or Dark2 are good for distinct categories)
# 'tab10' or 'tab20' are excellent for up to 10 or 20 distinct categories
cmap = plt.get_cmap('tab10') 
plate_colors = [cmap(i) for i in np.linspace(0, 1, num_plates)]
plate_color_map = {plate: plate_colors[i] for i, plate in enumerate(plate_list)}

# --- 2. Configuration ---
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

fig, axes = plt.subplots(2, 3, figsize=(28, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Running UMAP for {config['name']}...")
    
    X_subset = wells_df[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    texts = []
    
    # 3. Plot Plate by Plate
    for plate in plate_list:
        mask = wells_df['Plate'] == plate
        plate_embed = embedding[mask]
        plate_meta = wells_df[mask]
        
        is_ctrl = plate_meta[TREATMENT_COL] == "no_sgRNA"
        
        # Plot Mutants (Filled Circles)
        ax.scatter(plate_embed[~is_ctrl, 0], plate_embed[~is_ctrl, 1], 
                   color=plate_color_map[plate], marker='o', s=60, alpha=0.6, 
                   label=f"{plate} (Mutant)")
        
        # Plot Controls (Open Squares)
        ax.scatter(plate_embed[is_ctrl, 0], plate_embed[is_ctrl, 1], 
                   edgecolors=plate_color_map[plate], facecolors='none', 
                   marker='s', s=130, linewidths=2, label=f"{plate} (Control)")

        # 4. Add labels
        for i, row in enumerate(plate_meta.itertuples()):
            label = f"{row.Treatment}_{row.Plate.split('_')[-1]}"
            texts.append(ax.text(plate_embed[i, 0], plate_embed[i, 1], label, 
                                 fontsize=6, color=plate_color_map[plate]))

    # Formatting
    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')
    
    # Move legend to the side if it gets too crowded with 6+ plates
    if idx == 0:
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), markerscale=1, fontsize=8, ncol=1)
    
    # Adjust text to prevent overlapping labels
    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='gray', alpha=0.3))

plt.suptitle("Combined UMAP: Multi-Plate Analysis", fontsize=24, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from adjustText import adjust_text
import matplotlib.cm as cm
import numpy as np
PLATES = ["PLATE2_T0_masking","PLATE2_T1_masking","PLATE2_T2_masking"] 

# --- 1. Dynamic Color Setup ---
plate_list = sorted(wells_df['Plate'].unique())
num_plates = len(plate_list)

# Use a qualitative colormap (Set1, Set3, or Dark2 are good for distinct categories)
# 'tab10' or 'tab20' are excellent for up to 10 or 20 distinct categories
cmap = plt.get_cmap('tab10') 
plate_colors = [cmap(i) for i in np.linspace(0, 1, num_plates)]
plate_color_map = {plate: plate_colors[i] for i, plate in enumerate(plate_list)}

# --- 2. Configuration ---
feature_cols = [i for i in range(NUM_CHANNELS * FEATS_PER_CH)]
plot_configs = [
    {"name": "All Channels", "indices": feature_cols},
    {"name": "Channel 1", "indices": list(range(0, 1280))},
    {"name": "Channel 2", "indices": list(range(1280, 2560))},
    {"name": "Channel 3", "indices": list(range(2560, 3840))},
    {"name": "Channel 4", "indices": list(range(3840, 5120))},
    {"name": "Channel 5", "indices": list(range(5120, 6400))}
]

fig, axes = plt.subplots(2, 3, figsize=(28, 18))
axes = axes.flatten()

for idx, config in enumerate(plot_configs):
    ax = axes[idx]
    print(f"Running UMAP for {config['name']}...")
    
    X_subset = wells_df[config['indices']].values
    X_scaled = StandardScaler().fit_transform(X_subset)
    
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
    embedding = reducer.fit_transform(X_scaled)
    
    texts = []
    
    # 3. Plot Plate by Plate
    for plate in plate_list:
        mask = wells_df['Plate'] == plate
        plate_embed = embedding[mask]
        plate_meta = wells_df[mask]
        
        is_ctrl = plate_meta[TREATMENT_COL] == "no_sgRNA"
        
        # Plot Mutants (Filled Circles)
        ax.scatter(plate_embed[~is_ctrl, 0], plate_embed[~is_ctrl, 1], 
                   color=plate_color_map[plate], marker='o', s=60, alpha=0.6, 
                   label=f"{plate} (Mutant)")
        
        # Plot Controls (Open Squares)
        ax.scatter(plate_embed[is_ctrl, 0], plate_embed[is_ctrl, 1], 
                   edgecolors=plate_color_map[plate], facecolors='none', 
                   marker='s', s=130, linewidths=2, label=f"{plate} (Control)")

        # 4. Add labels
        for i, row in enumerate(plate_meta.itertuples()):
            label = f"{row.Treatment}_{row.Plate.split('_')[-1]}"
            texts.append(ax.text(plate_embed[i, 0], plate_embed[i, 1], label, 
                                 fontsize=6, color=plate_color_map[plate]))

    # Formatting
    ax.set_title(f"{config['name']}", fontsize=16, fontweight='bold')
    
    # Move legend to the side if it gets too crowded with 6+ plates
    if idx == 0:
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), markerscale=1, fontsize=8, ncol=1)
    
    # Adjust text to prevent overlapping labels
    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='-', color='gray', alpha=0.3))

plt.suptitle("Combined UMAP: Multi-Plate Analysis", fontsize=24, y=1.02)
plt.tight_layout()
plt.show()